# 群馬銀行モデルインプット用データ加工

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'IPAexGothic'
import seaborn as sns
import re
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
import japanize_matplotlib
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 2000

In [2]:
import seaborn as sns
sns.set()
sns.set('talk', 'whitegrid', 'dark', font_scale=1.5, font='IPAexGothic')

In [3]:
# 末尾に"/"を付与すること
PATH_INPUT = '../data/processed_raw/'
PATH_OUTPUT = '../data/model_input/'

## 集計メソッド

In [4]:
def godcustcd_preprocessing(df):
    '''入力データのGODCUSTCDについての前処理'''
    df = df[df['GODCUSTCD'].isnull()==False]
    df['GODCUSTCD'] = df['GODCUSTCD'].astype(np.int64)
    return df

In [5]:
def apply_suffix(df, period_length: int, suffix: str):
    '''各特徴量の0~11カラムにsuffixを付与'''
    col_names = list(df.iloc[:,0:2].columns)
    suffix_cols = [ str(i)+'_'+suffix for i in range(period_length) ]
    col_names.extend(suffix_cols)
    df.columns = col_names
    return df

In [6]:
def apply_suffix_tekiyo(df, suffix: str):
    fixed_col_names = list(df.iloc[:,0:2].columns)
    app_col_names = list(df.iloc[:,2:].columns)
    suffix_cols = [ str(i)+'_'+suffix for i in app_col_names ]
    fixed_col_names.extend(suffix_cols)
    df.columns = fixed_col_names
    return df

In [7]:
def get_monthly_yokomochi(df_base, df_source, sum_key, tar_item, pay_or_receive):
    '''入出金データの月次での横持ち化'''
    for tar_year in np.sort(df_source[sum_key].unique()):
        df_part = df_source[(df_source[sum_key]==tar_year)]
        df_part.rename(columns={tar_item:'{0}_{1}_{2}'.format(pay_or_receive, tar_item, tar_year)}, inplace=True)
        df_base = pd.merge(df_base, df_part.loc[:,list(CUST_KEY)+['{0}_{1}_{2}'.format(pay_or_receive, tar_item, tar_year)]],
                           on=list(CUST_KEY), how='left')
    return df_base

In [8]:
def get_zandaka_monthly_yokomochi(df_base, df_source, sum_key, tar_item):
    '''残高データの月次での横持ち化'''
    for tar_year in np.sort(df_source[sum_key].unique()):
        df_part = df_source[(df_source[sum_key]==tar_year)]
        df_part.rename(columns={tar_item:'{0}_{1}'.format(tar_item, tar_year)}, inplace=True)
        df_base = pd.merge(df_base, df_part.loc[:,list(CUST_KEY)+['{0}_{1}'.format(tar_item, tar_year)]],
                           on=list(CUST_KEY), how='left')
    return df_base

In [9]:
def get_monthly_period(df_base, tar_months, period_length, fill_null=False):
    '''期間ごとのデータ作成'''
    L = len(list(CUST_KEY))
    df_monthly_period = pd.DataFrame()
    for i in range(len(tar_months) - period_length):
        df_monthly_part = pd.concat([df_base[CUST_KEY], df_base.iloc[:,L+i:period_length+i+L]], axis=1)

        rename_cols = { df_monthly_part.columns[i+L] : i for i in range(len(df_monthly_part.columns)-L) }
        df_monthly_part.rename(columns=rename_cols, inplace=True)
        df_monthly_part.insert(L, 'YYYYMM', tar_months[i+period_length-1])
        
        df_monthly_period = pd.concat([df_monthly_period, df_monthly_part])
        if(fill_null==True):
            df_monthly_period.fillna(0, inplace=True)
    return df_monthly_period

In [10]:
def get_monthly_period_by_tekiyo(df_nyushukkin, kigyo_key='GODCUSTCD', tekiyo_key='TEKIYO', date_key='YYYYMM'):
    '''期間ごとのデータ作成（摘要込み版）'''
    tekiyo_name = list(df_nyushukkin[tekiyo_key].unique())
    df_nyushukkin_dic = {}
    
    for tekiyo in tekiyo_name:
        rename_dic = { i : str(i)+'_'+tekiyo for i in df_nyushukkin.columns[3:]}
        df_nyushukkin_sub = df_nyushukkin[df_nyushukkin[tekiyo_key]==tekiyo]
        df_nyushukkin_dic[tekiyo] = df_nyushukkin_sub.rename(columns=rename_dic)
        df_nyushukkin_dic[tekiyo] = df_nyushukkin_sub
    df_return = df_nyushukkin[[kigyo_key,date_key]].drop_duplicates()
    
    for tekiyo in tekiyo_name:
        df_nyushukkin_dic_tmp = df_nyushukkin_dic[tekiyo].drop(tekiyo_key, axis=1).copy()
        df_nyushukkin_dic_tmp = apply_suffix(df_nyushukkin_dic_tmp, 12, tekiyo)
        df_nyushukkin_dic_tmp[tekiyo+'_MEAN'] = df_nyushukkin_dic_tmp.iloc[:,2:].mean(axis='columns')
        df_return = pd.merge(df_return, df_nyushukkin_dic_tmp, on=[kigyo_key,date_key], how='left')
        
    df_return.fillna(0, inplace=True)
    return df_return

In [11]:
def year_aggregation(df_amount, df_count, period_length, months, cust_key, l):
    '''年次平均値を算出'''
    df_result = pd.DataFrame()
    for df_monthly in [df_amount, df_count]:
        df_year_part = get_monthly_period(df_monthly, months, period_length, True)
        sum_col_name = str.join('', re.findall('[a-zA-Z_]', df_monthly.columns[l+1])) + 'MEAN' + str(period_length)
        df_year_part[sum_col_name] = df_year_part.iloc[:,l+1:].mean(axis=1)
        df_year_part = df_year_part.loc[:,list(cust_key)+['YYYYMM',sum_col_name]]
        
        if(len(df_result) == 0):
            df_result = df_year_part
        else:
            df_result = pd.merge(df_result, df_year_part, on=list(cust_key)+['YYYYMM',], how='inner')
    return df_result

In [12]:
def year_aggregation_simple(df_monthly, period_length, months, cust_key, l):
    '''年次平均値を算出'''
    df_year_part = get_monthly_period(df_monthly, months, period_length, True)
    sum_col_name = str.join('', re.findall('[a-zA-Z_]', df_monthly.columns[l+1])) + 'MEAN' + str(period_length)
    df_year_part[sum_col_name] = df_year_part.iloc[:,l+1:].mean(axis=1)
    df_year_part = df_year_part.loc[:,list(cust_key)+['YYYYMM',sum_col_name]]
    return df_year_part

In [13]:
def normalization(df):
    '''正規化'''
    df_normalized = df.iloc[:,2:].apply(lambda x: x / np.max(x), axis=1)
    df_result = pd.concat([df.iloc[:,:2], df_normalized], axis=1)
    return df_result

In [14]:
def standardization(df):
    '''標準化'''
    df_standardized = df.iloc[:,2:].apply(lambda x: (x - np.mean(x)) / np.std(x), axis=1)
    df_result = pd.concat([df.iloc[:,:2], df_standardized], axis=1)
    return df_result

## 月次入出金データの取得

In [15]:
# 入金データの取得
df_nyukin = godcustcd_preprocessing(pd.read_csv(PATH_INPUT+'nyukin_yokin_monthly.csv', index_col=0))

In [16]:
print(df_nyukin.shape)
df_nyukin.sort_values(['GODCUSTCD','YYYYMM']).head()

(83700, 5)


,GODCUSTCD,YYYYMM,AMOUNT,COUNT,PAY_OR_RECEIVE
0,6316050371,201404,160952186,101,1
1,6316050371,201405,144256677,63,1
2,6316050371,201406,46509489,60,1
3,6316050371,201407,148835338,57,1
4,6316050371,201408,110489939,79,1


In [17]:
# 出金データの取得
df_shukkin = godcustcd_preprocessing(pd.read_csv(PATH_INPUT+'shukkin_yokin_monthly.csv', index_col=0))

In [18]:
print(df_shukkin.shape)
df_shukkin.sort_values(['GODCUSTCD','YYYYMM']).head()

(95181, 5)


,GODCUSTCD,YYYYMM,AMOUNT,COUNT,PAY_OR_RECEIVE
0,6316050371,201404,111715651,59,0
1,6316050371,201405,157794135,71,0
2,6316050371,201406,114105136,65,0
3,6316050371,201407,143943919,66,0
4,6316050371,201408,91085895,67,0


## 月次入出金データの横持ち化

In [19]:
CUST_KEY = ['GODCUSTCD'] 

In [20]:
# 　入金額横持ち
df_cust_master = pd.DataFrame(df_nyukin[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_nyukin_amount_monthly  = get_monthly_yokomochi(
    df_cust_master.copy(), df_nyukin[df_nyukin['PAY_OR_RECEIVE']==1], 'YYYYMM', 'AMOUNT', 'RECEIVE')

In [21]:
print(df_nyukin_amount_monthly.shape)
df_nyukin_amount_monthly.head()

(2383, 49)


,GODCUSTCD,RECEIVE_AMOUNT_201404,RECEIVE_AMOUNT_201405,RECEIVE_AMOUNT_201406,RECEIVE_AMOUNT_201407,RECEIVE_AMOUNT_201408,RECEIVE_AMOUNT_201409,RECEIVE_AMOUNT_201410,RECEIVE_AMOUNT_201411,RECEIVE_AMOUNT_201412,RECEIVE_AMOUNT_201501,RECEIVE_AMOUNT_201502,RECEIVE_AMOUNT_201503,RECEIVE_AMOUNT_201504,RECEIVE_AMOUNT_201505,RECEIVE_AMOUNT_201506,RECEIVE_AMOUNT_201507,RECEIVE_AMOUNT_201508,RECEIVE_AMOUNT_201509,RECEIVE_AMOUNT_201510,RECEIVE_AMOUNT_201511,RECEIVE_AMOUNT_201512,RECEIVE_AMOUNT_201601,RECEIVE_AMOUNT_201602,RECEIVE_AMOUNT_201603,RECEIVE_AMOUNT_201604,RECEIVE_AMOUNT_201605,RECEIVE_AMOUNT_201606,RECEIVE_AMOUNT_201607,RECEIVE_AMOUNT_201608,RECEIVE_AMOUNT_201609,RECEIVE_AMOUNT_201610,RECEIVE_AMOUNT_201611,RECEIVE_AMOUNT_201612,RECEIVE_AMOUNT_201701,RECEIVE_AMOUNT_201702,RECEIVE_AMOUNT_201703,RECEIVE_AMOUNT_201704,RECEIVE_AMOUNT_201705,RECEIVE_AMOUNT_201706,RECEIVE_AMOUNT_201707,RECEIVE_AMOUNT_201708,RECEIVE_AMOUNT_201709,RECEIVE_AMOUNT_201710,RECEIVE_AMOUNT_201711,RECEIVE_AMOUNT_201712,RECEIVE_AMOUNT_201801,RECEIVE_AMOUNT_201802,RECEIVE_AMOUNT_201803
0,6316050371,1.609522e+08,1.442567e+08,4.650949e+07,1.488353e+08,1.104899e+08,1.342007e+08,1.766987e+08,1.877868e+08,1.478777e+08,1.498025e+08,4.646728e+07,1.291094e+08,1.680629e+08,1.726335e+08,9.918190e+07,1.735985e+08,78793088.0,4.173174e+08,2.092626e+08,2.192083e+07,9.524645e+07,4.789399e+07,9.751388e+07,3.045394e+08,275088640.0,2.937388e+08,1.186921e+08,1.860852e+08,27459367.0,49064335.0,8.421147e+07,1.179600e+08,4.805246e+07,1.601947e+08,8.055420e+07,1.800172e+08,1.159650e+08,1.313244e+08,2.121545e+08,1.691488e+08,5.954699e+07,3.251439e+08,4.794050e+07,7.004947e+07,7.086751e+07,8.916649e+07,7.476910e+07,2.686801e+08
1,6316050390,1.299387e+08,3.305081e+08,3.957848e+08,1.303706e+08,1.915690e+08,2.324051e+08,1.073291e+08,1.414672e+08,2.395045e+08,3.369461e+08,5.200510e+08,1.904705e+08,1.859487e+08,3.954679e+08,2.590356e+08,2.981852e+08,199111769.0,2.473711e+08,2.722764e+08,3.122514e+08,1.320095e+08,1.507990e+08,1.888304e+08,2.473163e+08,121930331.0,2.039915e+08,9.203920e+07,1.044819e+08,441801227.0,336263887.0,1.469910e+08,2.042221e+08,2.824017e+08,3.867324e+08,2.884648e+08,1.377556e+08,8.688698e+07,2.078516e+08,9.961024e+07,5.123984e+08,4.692463e+08,1.491126e+08,2.503256e+08,1.270436e+08,2.506738e+08,3.763669e+08,2.982121e+08,2.146549e+08
2,6316050420,2.317329e+08,2.529290e+08,1.887685e+08,1.967674e+08,1.648066e+08,1.533848e+08,1.780142e+08,1.454117e+08,2.315770e+08,2.739177e+08,2.719888e+08,2.658429e+08,2.305907e+08,2.000686e+08,1.990247e+08,2.031478e+08,195078921.0,1.908092e+08,2.024102e+08,1.682495e+08,2.020586e+08,1.897555e+08,2.084883e+08,1.859367e+08,151865938.0,1.715869e+08,1.461078e+08,9.165330e+07,186581622.0,136530481.0,1.149111e+08,1.379675e+08,1.844324e+08,1.608791e+08,1.451869e+08,1.935479e+08,1.602606e+08,1.772221e+08,1.495851e+08,2.026654e+08,1.234979e+08,1.247310e+08,1.281536e+08,1.326888e+08,1.532076e+08,2.184700e+08,1.884979e+08,2.050122e+08
3,6316050622,2.010568e+09,1.649772e+09,1.972697e+09,1.740326e+09,1.504295e+09,1.706275e+09,1.558641e+09,2.170375e+09,1.740507e+09,1.557151e+09,2.095373e+09,1.956115e+09,1.530017e+09,1.212572e+09,1.476414e+09,1.307550e+09,969060157.0,1.139030e+09,1.027402e+09,1.735444e+09,1.385461e+09,1.220299e+09,1.218271e+09,1.242384e+09,982402169.0,1.204549e+09,1.321938e+09,1.236732e+09,959906171.0,930965255.0,1.038162e+09,1.722252e+09,1.597962e+09,1.511587e+09,1.585892e+09,1.690569e+09,1.411079e+09,1.408088e+09,1.429198e+09,1.120071e+09,1.160772e+09,1.292051e+09,1.482464e+09,2.157859e+09,2.062956e+09,2.167061e+09,1.891818e+09,1.807832e+09
4,6316050673,9.700000e+05,9.400000e+05,9.350000e+05,9.500000e+05,9.300060e+05,9.500000e+05,9.500000e+05,9.000000e+05,9.400000e+05,9.200000e+05,9.000150e+05,9.400000e+05,9.300000e+05,9.000000e+05,9.700000e+05,8.400000e+05,950010.0,9.000000e+05,1.000000e+06,8.000000e+05,9.000000e+05,9.000000e+05,9.000190e+05,8.500000e+05,1000000.0,8.000000e+05,8.500000e+05,8.800000e+05,2483004.0,NaN,2.500000e+05,9.00000

In [22]:
# 入金件数横持ち
df_nyukin_count_monthly  = get_monthly_yokomochi(
    df_cust_master.copy(), df_nyukin[df_nyukin['PAY_OR_RECEIVE']==1], 'YYYYMM', 'COUNT', 'RECEIVE')

In [23]:
print(df_nyukin_count_monthly.shape)
df_nyukin_count_monthly.head()

(2383, 49)


,GODCUSTCD,RECEIVE_COUNT_201404,RECEIVE_COUNT_201405,RECEIVE_COUNT_201406,RECEIVE_COUNT_201407,RECEIVE_COUNT_201408,RECEIVE_COUNT_201409,RECEIVE_COUNT_201410,RECEIVE_COUNT_201411,RECEIVE_COUNT_201412,RECEIVE_COUNT_201501,RECEIVE_COUNT_201502,RECEIVE_COUNT_201503,RECEIVE_COUNT_201504,RECEIVE_COUNT_201505,RECEIVE_COUNT_201506,RECEIVE_COUNT_201507,RECEIVE_COUNT_201508,RECEIVE_COUNT_201509,RECEIVE_COUNT_201510,RECEIVE_COUNT_201511,RECEIVE_COUNT_201512,RECEIVE_COUNT_201601,RECEIVE_COUNT_201602,RECEIVE_COUNT_201603,RECEIVE_COUNT_201604,RECEIVE_COUNT_201605,RECEIVE_COUNT_201606,RECEIVE_COUNT_201607,RECEIVE_COUNT_201608,RECEIVE_COUNT_201609,RECEIVE_COUNT_201610,RECEIVE_COUNT_201611,RECEIVE_COUNT_201612,RECEIVE_COUNT_201701,RECEIVE_COUNT_201702,RECEIVE_COUNT_201703,RECEIVE_COUNT_201704,RECEIVE_COUNT_201705,RECEIVE_COUNT_201706,RECEIVE_COUNT_201707,RECEIVE_COUNT_201708,RECEIVE_COUNT_201709,RECEIVE_COUNT_201710,RECEIVE_COUNT_201711,RECEIVE_COUNT_201712,RECEIVE_COUNT_201801,RECEIVE_COUNT_201802,RECEIVE_COUNT_201803
0,6316050371,101.0,63.0,60.0,57.0,79.0,80.0,72.0,70.0,61.0,53.0,67.0,80.0,86.0,65.0,56.0,52.0,69.0,60.0,63.0,63.0,62.0,51.0,54.0,70.0,89.0,55.0,49.0,45.0,56.0,56.0,49.0,60.0,45.0,59.0,54.0,54.0,70.0,48.0,40.0,49.0,60.0,64.0,49.0,57.0,55.0,65.0,50.0,60.0
1,6316050390,58.0,67.0,80.0,62.0,60.0,72.0,65.0,59.0,70.0,53.0,63.0,64.0,61.0,55.0,64.0,64.0,62.0,62.0,44.0,75.0,60.0,58.0,60.0,68.0,56.0,66.0,58.0,50.0,59.0,61.0,51.0,45.0,58.0,58.0,45.0,62.0,42.0,60.0,53.0,56.0,62.0,47.0,65.0,60.0,63.0,59.0,52.0,58.0
2,6316050420,340.0,319.0,347.0,322.0,299.0,328.0,321.0,290.0,367.0,312.0,320.0,363.0,326.0,311.0,341.0,322.0,321.0,317.0,316.0,305.0,336.0,319.0,354.0,361.0,331.0,352.0,345.0,308.0,333.0,321.0,327.0,306.0,333.0,327.0,329.0,349.0,324.0,317.0,322.0,316.0,316.0,296.0,321.0,302.0,378.0,359.0,369.0,386.0
3,6316050622,914.0,817.0,925.0,865.0,827.0,873.0,883.0,822.0,972.0,936.0,896.0,957.0,910.0,815.0,922.0,873.0,859.0,838.0,824.0,927.0,902.0,886.0,929.0,940.0,838.0,905.0,877.0,833.0,897.0,813.0,859.0,889.0,940.0,926.0,870.0,916.0,827.0,925.0,909.0,877.0,904.0,802.0,895.0,884.0,906.0,953.0,933.0,867.0
4,6316050673,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,NaN,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,NaN,1.0,1.0,2.0,1.0


In [24]:
# 出金額横持ち
df_cust_master = pd.DataFrame(df_shukkin[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_shukkin_amount_monthly = get_monthly_yokomochi(
    df_cust_master.copy(), df_shukkin[df_shukkin['PAY_OR_RECEIVE']==0], 'YYYYMM', 'AMOUNT', 'PAY')

In [25]:
print(df_shukkin_amount_monthly.shape)
df_shukkin_amount_monthly.head()

(2392, 49)


,GODCUSTCD,PAY_AMOUNT_201404,PAY_AMOUNT_201405,PAY_AMOUNT_201406,PAY_AMOUNT_201407,PAY_AMOUNT_201408,PAY_AMOUNT_201409,PAY_AMOUNT_201410,PAY_AMOUNT_201411,PAY_AMOUNT_201412,PAY_AMOUNT_201501,PAY_AMOUNT_201502,PAY_AMOUNT_201503,PAY_AMOUNT_201504,PAY_AMOUNT_201505,PAY_AMOUNT_201506,PAY_AMOUNT_201507,PAY_AMOUNT_201508,PAY_AMOUNT_201509,PAY_AMOUNT_201510,PAY_AMOUNT_201511,PAY_AMOUNT_201512,PAY_AMOUNT_201601,PAY_AMOUNT_201602,PAY_AMOUNT_201603,PAY_AMOUNT_201604,PAY_AMOUNT_201605,PAY_AMOUNT_201606,PAY_AMOUNT_201607,PAY_AMOUNT_201608,PAY_AMOUNT_201609,PAY_AMOUNT_201610,PAY_AMOUNT_201611,PAY_AMOUNT_201612,PAY_AMOUNT_201701,PAY_AMOUNT_201702,PAY_AMOUNT_201703,PAY_AMOUNT_201704,PAY_AMOUNT_201705,PAY_AMOUNT_201706,PAY_AMOUNT_201707,PAY_AMOUNT_201708,PAY_AMOUNT_201709,PAY_AMOUNT_201710,PAY_AMOUNT_201711,PAY_AMOUNT_201712,PAY_AMOUNT_201801,PAY_AMOUNT_201802,PAY_AMOUNT_201803
0,6316050371,1.117157e+08,1.577941e+08,1.141051e+08,1.439439e+08,9.108590e+07,1.489562e+08,1.920050e+08,1.256723e+08,7.915876e+07,2.455918e+08,6.844075e+07,1.130893e+08,1.737878e+08,1.547448e+08,1.193231e+08,1.523060e+08,1.001183e+08,385502489.0,9.640517e+07,7.162108e+07,9.888050e+07,5.552836e+07,9.320359e+07,2.395542e+08,312406096.0,1.549892e+08,2.666330e+08,113349475.0,1.540905e+08,126459956.0,7.537690e+07,4.976412e+07,1.186441e+08,1.404619e+08,6.861627e+07,1.599583e+08,1.382236e+08,1.457610e+08,1.827646e+08,1.379551e+08,1.245115e+08,1.684191e+08,2.145801e+08,5.330757e+07,6.731807e+07,9.447622e+07,7.848663e+07,1.620843e+08
1,6316050390,1.210884e+08,3.327815e+08,3.606182e+08,1.844980e+08,1.410058e+08,2.352854e+08,1.630746e+08,1.437948e+08,2.301050e+08,2.569568e+08,5.392552e+08,2.157648e+08,1.852647e+08,3.973916e+08,2.672336e+08,2.744699e+08,2.043940e+08,208981233.0,3.320173e+08,3.175790e+08,1.147046e+08,1.901923e+08,2.110922e+08,2.398883e+08,100050119.0,1.474310e+08,9.995155e+07,138824843.0,1.720995e+08,482080361.0,2.510648e+08,2.030583e+08,2.684286e+08,1.949750e+08,4.990655e+08,1.433000e+08,1.058109e+08,2.180570e+08,1.152946e+08,1.405932e+08,7.748797e+08,1.659451e+08,2.880296e+08,1.121526e+08,1.829530e+08,3.806666e+08,2.602944e+08,2.699917e+08
2,6316050420,2.431669e+08,2.285721e+08,2.080904e+08,1.967062e+08,1.646936e+08,1.859269e+08,1.580464e+08,1.529581e+08,2.039042e+08,2.494661e+08,2.562472e+08,2.584768e+08,2.274885e+08,2.046422e+08,2.192411e+08,1.944685e+08,2.180970e+08,182787526.0,1.855556e+08,2.182817e+08,1.836598e+08,1.798599e+08,2.037910e+08,1.704816e+08,146491959.0,1.602676e+08,1.338784e+08,112250931.0,1.972136e+08,125985584.0,1.087924e+08,1.213614e+08,1.685133e+08,1.636025e+08,1.510544e+08,1.624073e+08,1.505878e+08,1.739115e+08,1.429290e+08,1.938485e+08,1.502484e+08,1.175252e+08,1.475118e+08,1.414299e+08,1.453557e+08,1.425386e+08,2.863855e+08,4.647980e+08
3,6316050622,1.973253e+09,1.720483e+09,2.051282e+09,1.786125e+09,1.517387e+09,1.568158e+09,1.614584e+09,2.170172e+09,1.737495e+09,1.492166e+09,2.122193e+09,1.517464e+09,2.021787e+09,1.253656e+09,1.436268e+09,1.244928e+09,1.059508e+09,913707786.0,1.179451e+09,1.786793e+09,1.405414e+09,1.096449e+09,1.330254e+09,1.160521e+09,973414171.0,1.139140e+09,1.470830e+09,987969696.0,1.237550e+09,856736399.0,1.179944e+09,1.751017e+09,1.410371e+09,1.631144e+09,1.614766e+09,1.514744e+09,1.477303e+09,1.467457e+09,1.409530e+09,1.168946e+09,1.179828e+09,1.189610e+09,1.601755e+09,2.143473e+09,1.794201e+09,2.133990e+09,1.884936e+09,1.691695e+09
4,6316050673,9.722550e+05,9.371390e+05,9.347800e+05,9.401330e+05,9.376950e+05,9.428120e+05,9.397030e+05,9.229850e+05,9.352630e+05,9.182680e+05,9.015870e+05,9.347970e+05,9.114600e+05,9.157580e+05,9.201670e+05,8.973470e+05,9.084450e+05,906008.0,9.078380e+05,8.946790e+05,8.986950e+05,8.962570e+05,8.813850e+05,8.913820e+05,898324.0,8.952400e+05,8.716710e+05,882681.0,8.849530e+05,870615.0,9.456220e+05,9.333780e+05,9.293260e+05,9.356140e+05,9.215010e+05,9.226660e+05,9.377770e+05,9.272760e+05,9.186140e+05,9.218270e+05,9.203840e+05,9.167510e+05,4.996509e+07,9.264880e+05,9.331230e+05,9.342800e+0

In [26]:
# 出金件数横持ち
df_cust_master = pd.DataFrame(df_shukkin[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_shukkin_count_monthly = get_monthly_yokomochi(
    df_cust_master.copy(), df_shukkin[df_shukkin['PAY_OR_RECEIVE']==0], 'YYYYMM', 'COUNT', 'PAY')

In [27]:
print(df_shukkin_count_monthly.shape)
df_shukkin_count_monthly.head()

(2392, 49)


,GODCUSTCD,PAY_COUNT_201404,PAY_COUNT_201405,PAY_COUNT_201406,PAY_COUNT_201407,PAY_COUNT_201408,PAY_COUNT_201409,PAY_COUNT_201410,PAY_COUNT_201411,PAY_COUNT_201412,PAY_COUNT_201501,PAY_COUNT_201502,PAY_COUNT_201503,PAY_COUNT_201504,PAY_COUNT_201505,PAY_COUNT_201506,PAY_COUNT_201507,PAY_COUNT_201508,PAY_COUNT_201509,PAY_COUNT_201510,PAY_COUNT_201511,PAY_COUNT_201512,PAY_COUNT_201601,PAY_COUNT_201602,PAY_COUNT_201603,PAY_COUNT_201604,PAY_COUNT_201605,PAY_COUNT_201606,PAY_COUNT_201607,PAY_COUNT_201608,PAY_COUNT_201609,PAY_COUNT_201610,PAY_COUNT_201611,PAY_COUNT_201612,PAY_COUNT_201701,PAY_COUNT_201702,PAY_COUNT_201703,PAY_COUNT_201704,PAY_COUNT_201705,PAY_COUNT_201706,PAY_COUNT_201707,PAY_COUNT_201708,PAY_COUNT_201709,PAY_COUNT_201710,PAY_COUNT_201711,PAY_COUNT_201712,PAY_COUNT_201801,PAY_COUNT_201802,PAY_COUNT_201803
0,6316050371,59.0,71.0,65.0,66.0,67.0,65.0,64.0,67.0,69.0,56.0,61.0,66.0,67.0,70.0,65.0,67.0,66.0,67.0,62.0,60.0,65.0,62.0,64.0,62.0,62.0,67.0,66.0,62.0,66.0,62.0,61.0,53.0,59.0,56.0,56.0,59.0,61.0,67.0,65.0,60.0,63.0,57.0,63.0,61.0,60.0,63.0,59.0,65.0
1,6316050390,45.0,46.0,56.0,42.0,38.0,50.0,46.0,35.0,43.0,47.0,55.0,49.0,46.0,45.0,58.0,49.0,44.0,46.0,43.0,48.0,35.0,41.0,51.0,46.0,37.0,51.0,44.0,34.0,51.0,45.0,46.0,41.0,44.0,45.0,51.0,39.0,40.0,58.0,44.0,48.0,53.0,35.0,56.0,41.0,41.0,52.0,45.0,41.0
2,6316050420,71.0,70.0,82.0,77.0,63.0,75.0,62.0,58.0,75.0,63.0,64.0,69.0,73.0,63.0,85.0,66.0,68.0,64.0,54.0,71.0,66.0,62.0,66.0,60.0,60.0,80.0,70.0,64.0,76.0,59.0,64.0,60.0,64.0,69.0,63.0,67.0,58.0,79.0,67.0,72.0,68.0,57.0,68.0,66.0,61.0,80.0,77.0,75.0
3,6316050622,343.0,289.0,441.0,339.0,274.0,383.0,312.0,279.0,341.0,319.0,318.0,387.0,338.0,281.0,444.0,333.0,322.0,335.0,280.0,364.0,286.0,316.0,362.0,304.0,273.0,428.0,333.0,275.0,362.0,329.0,332.0,348.0,318.0,391.0,343.0,346.0,288.0,464.0,353.0,353.0,339.0,316.0,378.0,356.0,323.0,408.0,344.0,313.0
4,6316050673,5.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0


## 指定期間での分割横持ち化

In [28]:
period_length = 12
months = np.sort(df_nyukin['YYYYMM'].unique())

In [29]:
df_nyukin_amount_monthly_periods_forplot  = apply_suffix(
    get_monthly_period(df_nyukin_amount_monthly,  months, period_length, True), period_length, '預金_入金_金額')

In [30]:
print(df_nyukin_amount_monthly_periods_forplot.shape)
df_nyukin_amount_monthly_periods_forplot.head()

(85788, 14)


,GODCUSTCD,YYYYMM,0_預金_入金_金額,1_預金_入金_金額,2_預金_入金_金額,3_預金_入金_金額,4_預金_入金_金額,5_預金_入金_金額,6_預金_入金_金額,7_預金_入金_金額,8_預金_入金_金額,9_預金_入金_金額,10_預金_入金_金額,11_預金_入金_金額
0,6316050371,201503,1.609522e+08,1.442567e+08,4.650949e+07,1.488353e+08,1.104899e+08,1.342007e+08,1.766987e+08,1.877868e+08,1.478777e+08,1.498025e+08,4.646728e+07,1.291094e+08
1,6316050390,201503,1.299387e+08,3.305081e+08,3.957848e+08,1.303706e+08,1.915690e+08,2.324051e+08,1.073291e+08,1.414672e+08,2.395045e+08,3.369461e+08,5.200510e+08,1.904705e+08
2,6316050420,201503,2.317329e+08,2.529290e+08,1.887685e+08,1.967674e+08,1.648066e+08,1.533848e+08,1.780142e+08,1.454117e+08,2.315770e+08,2.739177e+08,2.719888e+08,2.658429e+08
3,6316050622,201503,2.010568e+09,1.649772e+09,1.972697e+09,1.740326e+09,1.504295e+09,1.706275e+09,1.558641e+09,2.170375e+09,1.740507e+09,1.557151e+09,2.095373e+09,1.956115e+09
4,6316050673,201503,9.700000e+05,9.400000e+05,9.350000e+05,9.500000e+05,9.300060e+05,9.500000e+05,9.500000e+05,9.000000e+05,9.400000e+05,9.200000e+05,9.000150e+05,9.400000e+05


In [31]:
df_nyukin_count_monthly_periods_forplot  = apply_suffix(
    get_monthly_period(df_nyukin_count_monthly,  months, period_length, True), period_length, '預金_入金_件数')

In [32]:
print(df_nyukin_count_monthly_periods_forplot.shape)
df_nyukin_count_monthly_periods_forplot.head()

(85788, 14)


,GODCUSTCD,YYYYMM,0_預金_入金_件数,1_預金_入金_件数,2_預金_入金_件数,3_預金_入金_件数,4_預金_入金_件数,5_預金_入金_件数,6_預金_入金_件数,7_預金_入金_件数,8_預金_入金_件数,9_預金_入金_件数,10_預金_入金_件数,11_預金_入金_件数
0,6316050371,201503,101.0,63.0,60.0,57.0,79.0,80.0,72.0,70.0,61.0,53.0,67.0,80.0
1,6316050390,201503,58.0,67.0,80.0,62.0,60.0,72.0,65.0,59.0,70.0,53.0,63.0,64.0
2,6316050420,201503,340.0,319.0,347.0,322.0,299.0,328.0,321.0,290.0,367.0,312.0,320.0,363.0
3,6316050622,201503,914.0,817.0,925.0,865.0,827.0,873.0,883.0,822.0,972.0,936.0,896.0,957.0
4,6316050673,201503,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0


In [33]:
df_shukkin_amount_monthly_periods_forplot = apply_suffix(
    get_monthly_period(df_shukkin_amount_monthly, months, period_length, True), period_length, '預金_出金_金額')

In [34]:
print(df_shukkin_amount_monthly_periods_forplot.shape)
df_shukkin_amount_monthly_periods_forplot.head()

(86112, 14)


,GODCUSTCD,YYYYMM,0_預金_出金_金額,1_預金_出金_金額,2_預金_出金_金額,3_預金_出金_金額,4_預金_出金_金額,5_預金_出金_金額,6_預金_出金_金額,7_預金_出金_金額,8_預金_出金_金額,9_預金_出金_金額,10_預金_出金_金額,11_預金_出金_金額
0,6316050371,201503,1.117157e+08,1.577941e+08,1.141051e+08,1.439439e+08,9.108590e+07,1.489562e+08,1.920050e+08,1.256723e+08,7.915876e+07,2.455918e+08,6.844075e+07,1.130893e+08
1,6316050390,201503,1.210884e+08,3.327815e+08,3.606182e+08,1.844980e+08,1.410058e+08,2.352854e+08,1.630746e+08,1.437948e+08,2.301050e+08,2.569568e+08,5.392552e+08,2.157648e+08
2,6316050420,201503,2.431669e+08,2.285721e+08,2.080904e+08,1.967062e+08,1.646936e+08,1.859269e+08,1.580464e+08,1.529581e+08,2.039042e+08,2.494661e+08,2.562472e+08,2.584768e+08
3,6316050622,201503,1.973253e+09,1.720483e+09,2.051282e+09,1.786125e+09,1.517387e+09,1.568158e+09,1.614584e+09,2.170172e+09,1.737495e+09,1.492166e+09,2.122193e+09,1.517464e+09
4,6316050673,201503,9.722550e+05,9.371390e+05,9.347800e+05,9.401330e+05,9.376950e+05,9.428120e+05,9.397030e+05,9.229850e+05,9.352630e+05,9.182680e+05,9.015870e+05,9.347970e+05


In [35]:
df_shukkin_count_monthly_periods_forplot = apply_suffix(
    get_monthly_period(df_shukkin_count_monthly, months, period_length, True), period_length, '預金_出金_件数')

In [36]:
print(df_shukkin_count_monthly_periods_forplot.shape)
df_shukkin_count_monthly_periods_forplot.head()

(86112, 14)


,GODCUSTCD,YYYYMM,0_預金_出金_件数,1_預金_出金_件数,2_預金_出金_件数,3_預金_出金_件数,4_預金_出金_件数,5_預金_出金_件数,6_預金_出金_件数,7_預金_出金_件数,8_預金_出金_件数,9_預金_出金_件数,10_預金_出金_件数,11_預金_出金_件数
0,6316050371,201503,59.0,71.0,65.0,66.0,67.0,65.0,64.0,67.0,69.0,56.0,61.0,66.0
1,6316050390,201503,45.0,46.0,56.0,42.0,38.0,50.0,46.0,35.0,43.0,47.0,55.0,49.0
2,6316050420,201503,71.0,70.0,82.0,77.0,63.0,75.0,62.0,58.0,75.0,63.0,64.0,69.0
3,6316050622,201503,343.0,289.0,441.0,339.0,274.0,383.0,312.0,279.0,341.0,319.0,318.0,387.0
4,6316050673,201503,5.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,2.0,2.0


## 正規化

In [37]:
df_nyukin_amount_monthly_periods_forplot_norm = normalization(df_nyukin_amount_monthly_periods_forplot)
print(df_nyukin_amount_monthly_periods_forplot_norm.shape)

(85788, 14)


In [ ]:
df_nyukin_amount_monthly_periods_forplot_norm.head()

In [38]:
df_shukkin_amount_monthly_periods_forplot_norm = normalization(df_shukkin_amount_monthly_periods_forplot)
print(df_shukkin_amount_monthly_periods_forplot_norm.shape)

(86112, 14)


In [39]:
df_nyukin_count_monthly_periods_forplot_norm = normalization(df_nyukin_count_monthly_periods_forplot)
print(df_nyukin_count_monthly_periods_forplot_norm.shape)

(85788, 14)


In [40]:
df_shukkin_count_monthly_periods_forplot_norm = normalization(df_shukkin_count_monthly_periods_forplot)
print(df_shukkin_count_monthly_periods_forplot_norm.shape)

(86112, 14)


## 標準化

In [41]:
df_nyukin_amount_monthly_periods_forplot_stand = standardization(df_nyukin_amount_monthly_periods_forplot)
print(df_nyukin_amount_monthly_periods_forplot_stand.shape)

(85788, 14)


In [42]:
df_shukkin_amount_monthly_periods_forplot_stand = standardization(df_shukkin_amount_monthly_periods_forplot)
print(df_shukkin_amount_monthly_periods_forplot_stand.shape)

(86112, 14)


In [43]:
df_nyukin_count_monthly_periods_forplot_stand = standardization(df_nyukin_count_monthly_periods_forplot)
print(df_nyukin_count_monthly_periods_forplot_stand.shape)

(85788, 14)


In [44]:
df_shukkin_count_monthly_periods_forplot_stand = standardization(df_shukkin_count_monthly_periods_forplot)
print(df_shukkin_count_monthly_periods_forplot_stand.shape)

(86112, 14)


## 年次集計値の作成

In [45]:
# 年次集計の設定値
period_length = 12
months = np.sort(df_shukkin['YYYYMM'].unique())
CUST_KEY = ['GODCUSTCD']
L = len(list(CUST_KEY))

In [46]:
# 入金データの年次集計値
df_nyukin_freq  = year_aggregation(
    df_nyukin_amount_monthly, df_nyukin_count_monthly,
    period_length, months, CUST_KEY, L
)
df_nyukin_freq = df_nyukin_freq.rename(columns=
                                       {'RECEIVE_AMOUNT_MEAN12':'YOKIN_RECEIVE_AMOUNT_MEAN12',
                                         'RECEIVE_COUNT_MEAN12':'YOKIN_RECEIVE_COUNT_MEAN12'})

In [47]:
print(df_nyukin_freq.shape)
df_nyukin_freq.head()

(85788, 4)


,GODCUSTCD,YYYYMM,YOKIN_RECEIVE_AMOUNT_MEAN12,YOKIN_RECEIVE_COUNT_MEAN12
0,6316050371,201503,1.319156e+08,70.250000
1,6316050390,201503,2.455287e+08,64.416667
2,6316050420,201503,2.129285e+08,327.333333
3,6316050622,201503,1.805175e+09,890.583333
4,6316050673,201503,9.354184e+05,1.166667


In [48]:
# # 入金データの年次集計値（正規化）
# df_nyukin_freq_norm  = year_aggregation(
#     df_nyukin_amount_monthly_norm, df_nyukin_count_monthly_norm,
#     period_length, months, CUST_KEY, L
# )
# df_nyukin_freq_norm = df_nyukin_freq_norm.rename(columns=
#                                                  {'RECEIVE_AMOUNT_MEAN12':'YOKIN_RECEIVE_AMOUNT_MEAN12',
#                                                   'RECEIVE_COUNT_MEAN12':'YOKIN_RECEIVE_COUNT_MEAN12'})

In [49]:
# print(df_nyukin_freq_norm.shape)
# df_nyukin_freq_norm.head()

In [50]:
# # 入金データの年次集計値（標準化）
# df_nyukin_freq_stand  = year_aggregation(
#     df_nyukin_amount_monthly_stand, df_nyukin_count_monthly_stand,
#     period_length, months, CUST_KEY, L
# )
# df_nyukin_freq_stand = df_nyukin_freq_stand.rename(columns=
#                                                  {'RECEIVE_AMOUNT_MEAN12':'YOKIN_RECEIVE_AMOUNT_MEAN12',
#                                                   'RECEIVE_COUNT_MEAN12':'YOKIN_RECEIVE_COUNT_MEAN12'})

In [51]:
# print(df_nyukin_freq_stand.shape)
# df_nyukin_freq_stand.head()

In [52]:
# 出金データの年次集計値
df_shukkin_freq  = year_aggregation(
    df_shukkin_amount_monthly, df_shukkin_count_monthly,
    period_length, months, CUST_KEY, L
)
df_shukkin_freq = df_shukkin_freq.rename(columns=
                                       {'PAY_AMOUNT_MEAN12':'YOKIN_PAY_AMOUNT_MEAN12',
                                         'PAY_COUNT_MEAN12':'YOKIN_PAY_COUNT_MEAN12'})

In [53]:
print(df_shukkin_freq.shape)
df_shukkin_freq.head()

(86112, 4)


,GODCUSTCD,YYYYMM,YOKIN_PAY_AMOUNT_MEAN12,YOKIN_PAY_COUNT_MEAN12
0,6316050371,201503,1.326299e+08,64.666667
1,6316050390,201503,2.436857e+08,46.000000
2,6316050420,201503,2.088546e+08,69.083333
3,6316050622,201503,1.772564e+09,335.416667
4,6316050673,201503,9.347848e+05,3.083333


In [54]:
# # 出金データの年次集計値（正規化）
# df_shukkin_freq_norm  = year_aggregation(
#     df_shukkin_amount_monthly_norm, df_shukkin_count_monthly_norm,
#     period_length, months, CUST_KEY, L
# )
# df_shukkin_freq_norm = df_shukkin_freq_norm.rename(columns=
#                                                    {'PAY_AMOUNT_MEAN12':'YOKIN_PAY_AMOUNT_MEAN12',
#                                                     'PAY_COUNT_MEAN12':'YOKIN_PAY_COUNT_MEAN12'})

In [55]:
# print(df_shukkin_freq_norm.shape)
# df_shukkin_freq_norm.head()

In [56]:
# # 出金データの年次集計値（標準化）
# df_shukkin_freq_stand  = year_aggregation(
#     df_shukkin_amount_monthly_stand, df_shukkin_count_monthly_stand,
#     period_length, months, CUST_KEY, L
# )
# df_shukkin_freq_stand = df_shukkin_freq_stand.rename(columns=
#                                                      {'PAY_AMOUNT_MEAN12':'YOKIN_PAY_AMOUNT_MEAN12',
#                                                       'PAY_COUNT_MEAN12':'YOKIN_PAY_COUNT_MEAN12'})

In [57]:
# print(df_shukkin_freq_stand.shape)
# df_shukkin_freq_stand.head()

## 月次入出金データ（摘要込み）の取得

In [58]:
# 入金データの取得
df_nyukin_tekiyo = godcustcd_preprocessing(pd.read_csv(PATH_INPUT+'nyukin_yokin_monthly_include_tekiyo.csv', index_col=0))

In [59]:
print(df_nyukin_tekiyo.shape)
df_nyukin_tekiyo.sort_values(['GODCUSTCD','YYYYMM']).head()

(92361, 6)


,GODCUSTCD,YYYYMM,TEKIYO,AMOUNT,COUNT,PAY_OR_RECEIVE
0,6316050371,201404,摘要なし,160952186,101,1
1,6316050371,201405,摘要なし,144256677,63,1
2,6316050371,201406,当行株式配当金,8469,1,1
3,6316050371,201406,摘要なし,46501020,59,1
4,6316050371,201407,摘要なし,148835338,57,1


In [60]:
# 出金データの取得
df_shukkin_tekiyo = godcustcd_preprocessing(pd.read_csv(PATH_INPUT+'shukkin_yokin_monthly_include_tekiyo.csv', index_col=0))

In [61]:
print(df_shukkin_tekiyo.shape)
df_shukkin_tekiyo.sort_values(['GODCUSTCD','YYYYMM']).head()

(330531, 6)


,GODCUSTCD,YYYYMM,TEKIYO,AMOUNT,COUNT,PAY_OR_RECEIVE
0,6316050371,201404,その他出金,13242874,28,0
1,6316050371,201404,ガス料,1575,1,0
2,6316050371,201404,クレジット,174995,6,0
3,6316050371,201404,摘要なし,98051858,19,0
4,6316050371,201404,群銀ＶＩＳＡ,239677,1,0


## 月次入出金データ（摘要込み）の横持ち化

In [62]:
CUST_KEY = ['GODCUSTCD','TEKIYO'] 

In [63]:
# 　入金額横持ち
df_cust_master = pd.DataFrame(df_nyukin_tekiyo[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_nyukin_tekiyo_amount_monthly  = get_monthly_yokomochi(
    df_cust_master.copy(), df_nyukin_tekiyo[df_nyukin_tekiyo['PAY_OR_RECEIVE']==1], 'YYYYMM', 'AMOUNT', 'RECEIVE')

In [64]:
print(df_nyukin_tekiyo_amount_monthly.shape)
df_nyukin_tekiyo_amount_monthly.head()

(2914, 50)


,GODCUSTCD,TEKIYO,RECEIVE_AMOUNT_201404,RECEIVE_AMOUNT_201405,RECEIVE_AMOUNT_201406,RECEIVE_AMOUNT_201407,RECEIVE_AMOUNT_201408,RECEIVE_AMOUNT_201409,RECEIVE_AMOUNT_201410,RECEIVE_AMOUNT_201411,RECEIVE_AMOUNT_201412,RECEIVE_AMOUNT_201501,RECEIVE_AMOUNT_201502,RECEIVE_AMOUNT_201503,RECEIVE_AMOUNT_201504,RECEIVE_AMOUNT_201505,RECEIVE_AMOUNT_201506,RECEIVE_AMOUNT_201507,RECEIVE_AMOUNT_201508,RECEIVE_AMOUNT_201509,RECEIVE_AMOUNT_201510,RECEIVE_AMOUNT_201511,RECEIVE_AMOUNT_201512,RECEIVE_AMOUNT_201601,RECEIVE_AMOUNT_201602,RECEIVE_AMOUNT_201603,RECEIVE_AMOUNT_201604,RECEIVE_AMOUNT_201605,RECEIVE_AMOUNT_201606,RECEIVE_AMOUNT_201607,RECEIVE_AMOUNT_201608,RECEIVE_AMOUNT_201609,RECEIVE_AMOUNT_201610,RECEIVE_AMOUNT_201611,RECEIVE_AMOUNT_201612,RECEIVE_AMOUNT_201701,RECEIVE_AMOUNT_201702,RECEIVE_AMOUNT_201703,RECEIVE_AMOUNT_201704,RECEIVE_AMOUNT_201705,RECEIVE_AMOUNT_201706,RECEIVE_AMOUNT_201707,RECEIVE_AMOUNT_201708,RECEIVE_AMOUNT_201709,RECEIVE_AMOUNT_201710,RECEIVE_AMOUNT_201711,RECEIVE_AMOUNT_201712,RECEIVE_AMOUNT_201801,RECEIVE_AMOUNT_201802,RECEIVE_AMOUNT_201803
0,6316050371,摘要なし,160952186.0,144256677.0,46501020.0,148835338.0,110488419.0,134200676.0,176698655.0,187786845.0,147870118.0,149802461.0,46465996.0,129109398.0,168062883.0,172633461.0,99170886.0,173598472.0,78791451.0,417317377.0,209262559.0,21920833.0,95237137.0,47893987.0,97512599.0,304539384.0,275088640.0,293738786.0,118681111.0,186085245.0,27457497.0,49064335.0,84211467.0,117959995.0,48042300.0,160194731.0,80552914.0,180017208.0,115965012.0,131324376.0,212144329.0,169148830.0,59545237.0,325143939.0,47940498.0,70049474.0,70857345.0,89166486.0,74767813.0,268680068.0
1,6316050371,当行株式配当金,NaN,NaN,8469.0,NaN,NaN,NaN,NaN,NaN,7622.0,NaN,NaN,NaN,NaN,NaN,11010.0,NaN,NaN,NaN,NaN,NaN,9316.0,NaN,NaN,NaN,NaN,NaN,11010.0,NaN,NaN,NaN,NaN,NaN,10163.0,NaN,NaN,NaN,NaN,NaN,10163.0,NaN,NaN,NaN,NaN,NaN,10163.0,NaN,NaN,NaN
2,6316050371,当行以外株式配当金,NaN,NaN,NaN,NaN,1520.0,NaN,NaN,NaN,NaN,NaN,1286.0,NaN,NaN,NaN,NaN,NaN,1637.0,NaN,NaN,NaN,NaN,NaN,1286.0,NaN,NaN,NaN,NaN,NaN,1870.0,NaN,NaN,NaN,NaN,NaN,1286.0,NaN,NaN,NaN,NaN,NaN,1753.0,NaN,NaN,NaN,NaN,NaN,1286.0,NaN
3,6316050390,摘要なし,129938704.0,330508064.0,395365634.0,130370584.0,191569002.0,232405088.0,107329130.0,141467193.0,239127269.0,336946065.0,520051023.0,190470469.0,185948666.0,395467941.0,258490692.0,298185207.0,199111769.0,247371083.0,272276425.0,312251416.0,131548360.0,150799049.0,188830389.0,247316272.0,121930331.0,203991469.0,91494253.0,104481923.0,441801227.0,336263887.0,146991043.0,204222104.0,281898626.0,386732356.0,288464789.0,137755582.0,86886977.0,207851633.0,99107208.0,512398408.0,469246276.0,149112584.0,250325648.0,127043563.0,250170721.0,376366866.0,298212103.0,214654929.0
4,6316050390,当行株式配当金,NaN,NaN,419191.0,NaN,NaN,NaN,NaN,NaN,377272.0,NaN,NaN,NaN,NaN,NaN,544948.0,NaN,NaN,NaN,NaN,NaN,461110.0,NaN,NaN,NaN,NaN,NaN,544948.0,NaN,NaN,NaN,NaN,NaN,503029.0,NaN,NaN,NaN,NaN,NaN,503029.0,NaN,NaN,NaN,NaN,NaN,503029.0,NaN,NaN,NaN


In [65]:
# 入金件数横持ち
df_nyukin_tekiyo_count_monthly  = get_monthly_yokomochi(
    df_cust_master.copy(), df_nyukin_tekiyo[df_nyukin_tekiyo['PAY_OR_RECEIVE']==1], 'YYYYMM', 'COUNT', 'RECEIVE')

In [66]:
print(df_nyukin_tekiyo_count_monthly.shape)
df_nyukin_tekiyo_count_monthly.head()

(2914, 50)


,GODCUSTCD,TEKIYO,RECEIVE_COUNT_201404,RECEIVE_COUNT_201405,RECEIVE_COUNT_201406,RECEIVE_COUNT_201407,RECEIVE_COUNT_201408,RECEIVE_COUNT_201409,RECEIVE_COUNT_201410,RECEIVE_COUNT_201411,RECEIVE_COUNT_201412,RECEIVE_COUNT_201501,RECEIVE_COUNT_201502,RECEIVE_COUNT_201503,RECEIVE_COUNT_201504,RECEIVE_COUNT_201505,RECEIVE_COUNT_201506,RECEIVE_COUNT_201507,RECEIVE_COUNT_201508,RECEIVE_COUNT_201509,RECEIVE_COUNT_201510,RECEIVE_COUNT_201511,RECEIVE_COUNT_201512,RECEIVE_COUNT_201601,RECEIVE_COUNT_201602,RECEIVE_COUNT_201603,RECEIVE_COUNT_201604,RECEIVE_COUNT_201605,RECEIVE_COUNT_201606,RECEIVE_COUNT_201607,RECEIVE_COUNT_201608,RECEIVE_COUNT_201609,RECEIVE_COUNT_201610,RECEIVE_COUNT_201611,RECEIVE_COUNT_201612,RECEIVE_COUNT_201701,RECEIVE_COUNT_201702,RECEIVE_COUNT_201703,RECEIVE_COUNT_201704,RECEIVE_COUNT_201705,RECEIVE_COUNT_201706,RECEIVE_COUNT_201707,RECEIVE_COUNT_201708,RECEIVE_COUNT_201709,RECEIVE_COUNT_201710,RECEIVE_COUNT_201711,RECEIVE_COUNT_201712,RECEIVE_COUNT_201801,RECEIVE_COUNT_201802,RECEIVE_COUNT_201803
0,6316050371,摘要なし,101.0,63.0,59.0,57.0,78.0,80.0,72.0,70.0,60.0,53.0,66.0,80.0,86.0,65.0,55.0,52.0,68.0,60.0,63.0,63.0,61.0,51.0,53.0,70.0,89.0,55.0,48.0,45.0,55.0,56.0,49.0,60.0,44.0,59.0,53.0,54.0,70.0,48.0,39.0,49.0,59.0,64.0,49.0,57.0,54.0,65.0,49.0,60.0
1,6316050371,当行株式配当金,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2,6316050371,当行以外株式配当金,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,6316050390,摘要なし,58.0,67.0,79.0,62.0,60.0,72.0,65.0,59.0,69.0,53.0,63.0,64.0,61.0,55.0,63.0,64.0,62.0,62.0,44.0,75.0,59.0,58.0,60.0,68.0,56.0,66.0,57.0,50.0,59.0,61.0,51.0,45.0,57.0,58.0,45.0,62.0,42.0,60.0,52.0,56.0,62.0,47.0,65.0,60.0,62.0,59.0,52.0,58.0
4,6316050390,当行株式配当金,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [67]:
# 出金額横持ち
df_cust_master = pd.DataFrame(df_shukkin_tekiyo[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_shukkin_tekiyo_amount_monthly = get_monthly_yokomochi(
    df_cust_master.copy(), df_shukkin_tekiyo[df_shukkin_tekiyo['PAY_OR_RECEIVE']==0], 'YYYYMM', 'AMOUNT', 'PAY')

In [68]:
print(df_shukkin_tekiyo_amount_monthly.shape)
df_shukkin_tekiyo_amount_monthly.head()

(9615, 50)


,GODCUSTCD,TEKIYO,PAY_AMOUNT_201404,PAY_AMOUNT_201405,PAY_AMOUNT_201406,PAY_AMOUNT_201407,PAY_AMOUNT_201408,PAY_AMOUNT_201409,PAY_AMOUNT_201410,PAY_AMOUNT_201411,PAY_AMOUNT_201412,PAY_AMOUNT_201501,PAY_AMOUNT_201502,PAY_AMOUNT_201503,PAY_AMOUNT_201504,PAY_AMOUNT_201505,PAY_AMOUNT_201506,PAY_AMOUNT_201507,PAY_AMOUNT_201508,PAY_AMOUNT_201509,PAY_AMOUNT_201510,PAY_AMOUNT_201511,PAY_AMOUNT_201512,PAY_AMOUNT_201601,PAY_AMOUNT_201602,PAY_AMOUNT_201603,PAY_AMOUNT_201604,PAY_AMOUNT_201605,PAY_AMOUNT_201606,PAY_AMOUNT_201607,PAY_AMOUNT_201608,PAY_AMOUNT_201609,PAY_AMOUNT_201610,PAY_AMOUNT_201611,PAY_AMOUNT_201612,PAY_AMOUNT_201701,PAY_AMOUNT_201702,PAY_AMOUNT_201703,PAY_AMOUNT_201704,PAY_AMOUNT_201705,PAY_AMOUNT_201706,PAY_AMOUNT_201707,PAY_AMOUNT_201708,PAY_AMOUNT_201709,PAY_AMOUNT_201710,PAY_AMOUNT_201711,PAY_AMOUNT_201712,PAY_AMOUNT_201801,PAY_AMOUNT_201802,PAY_AMOUNT_201803
0,6316050371,その他出金,13242874.0,11278738.0,24145073.0,12566812.0,11911535.0,15302422.0,13616765.0,12326121.0,21613660.0,12756140.0,13655479.0,15545042.0,13576856.0,31585379.0,74674912.0,13230106.0,16630124.0,62558956.0,10497740.0,15813355.0,17765719.0,13324442.0,20448036.0,66234602.0,10461788.0,17365003.0,91237220.0,18197845.0,19380485.0,13018876.0,19748020.0,10129254.0,17936850.0,10851280.0,11742556.0,12425976.0,11031801.0,31141928.0,25651174.0,13664467.0,13123024.0,10197006.0,22161406.0,11620516.0,21971478.0,11699917.0,12092432.0,13089845.0
1,6316050371,ガス料,1575.0,1575.0,1620.0,1620.0,1620.0,1620.0,1620.0,1620.0,1620.0,1620.0,2882.0,2120.0,2194.0,2094.0,1770.0,1620.0,1728.0,1728.0,1728.0,1728.0,1728.0,1728.0,1728.0,1728.0,2509.0,1728.0,1728.0,1728.0,1728.0,1728.0,1728.0,1728.0,1728.0,1879.0,2751.0,2556.0,1728.0,1728.0,1795.0,1728.0,1728.0,1728.0,1728.0,1620.0,1901.0,1728.0,3432.0,1728.0
2,6316050371,クレジット,174995.0,324802.0,127925.0,143466.0,258432.0,200285.0,127925.0,143466.0,168817.0,174395.0,164955.0,132275.0,319536.0,256431.0,131195.0,144576.0,277243.0,131195.0,203555.0,143075.0,1069076.0,174395.0,146736.0,119045.0,119045.0,310582.0,119045.0,189234.0,273636.0,237510.0,143129.0,146369.0,1500190.0,163263.0,151390.0,99290.0,285681.0,416834.0,274977.0,222376.0,365371.0,226405.0,287049.0,278788.0,1759032.0,349214.0,371432.0,421506.0
3,6316050371,摘要なし,98051858.0,145853219.0,89512878.0,130887368.0,78561627.0,133099327.0,177894583.0,112627605.0,57075931.0,232314243.0,54336770.0,96988637.0,159581829.0,122523545.0,44242967.0,138606661.0,82531257.0,322237797.0,85342039.0,55194781.0,79646747.0,41652110.0,71973995.0,172808454.0,301563152.0,136674829.0,174609323.0,94687289.0,133992700.0,112832574.0,54908628.0,39083977.0,98936905.0,128926272.0,56304098.0,147119783.0,126471447.0,113639754.0,156435524.0,123649050.0,110450412.0,157517759.0,191459994.0,40843407.0,43138516.0,81705976.0,65500008.0,148210185.0
4,6316050371,群銀ＶＩＳＡ,239677.0,331030.0,313340.0,339949.0,348417.0,347804.0,359159.0,568843.0,295258.0,341081.0,276615.0,416945.0,303256.0,373194.0,269080.0,319979.0,558461.0,463673.0,357239.0,251821.0,394269.0,265836.0,417456.0,278292.0,256762.0,419528.0,556013.0,270866.0,225414.0,249334.0,453594.0,296125.0,265335.0,302147.0,302635.0,200695.0,429877.0,351990.0,294770.0,289712.0,461323.0,472933.0,458492.0,455213.0,444195.0,506448.0,411618.0,357986.0


In [69]:
# 出金件数横持ち
df_cust_master = pd.DataFrame(df_shukkin_tekiyo[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_shukkin_tekiyo_count_monthly = get_monthly_yokomochi(
    df_cust_master.copy(), df_shukkin_tekiyo[df_shukkin_tekiyo['PAY_OR_RECEIVE']==0], 'YYYYMM', 'COUNT', 'PAY')

In [70]:
print(df_shukkin_tekiyo_count_monthly.shape)
df_shukkin_tekiyo_count_monthly.head()

(9615, 50)


,GODCUSTCD,TEKIYO,PAY_COUNT_201404,PAY_COUNT_201405,PAY_COUNT_201406,PAY_COUNT_201407,PAY_COUNT_201408,PAY_COUNT_201409,PAY_COUNT_201410,PAY_COUNT_201411,PAY_COUNT_201412,PAY_COUNT_201501,PAY_COUNT_201502,PAY_COUNT_201503,PAY_COUNT_201504,PAY_COUNT_201505,PAY_COUNT_201506,PAY_COUNT_201507,PAY_COUNT_201508,PAY_COUNT_201509,PAY_COUNT_201510,PAY_COUNT_201511,PAY_COUNT_201512,PAY_COUNT_201601,PAY_COUNT_201602,PAY_COUNT_201603,PAY_COUNT_201604,PAY_COUNT_201605,PAY_COUNT_201606,PAY_COUNT_201607,PAY_COUNT_201608,PAY_COUNT_201609,PAY_COUNT_201610,PAY_COUNT_201611,PAY_COUNT_201612,PAY_COUNT_201701,PAY_COUNT_201702,PAY_COUNT_201703,PAY_COUNT_201704,PAY_COUNT_201705,PAY_COUNT_201706,PAY_COUNT_201707,PAY_COUNT_201708,PAY_COUNT_201709,PAY_COUNT_201710,PAY_COUNT_201711,PAY_COUNT_201712,PAY_COUNT_201801,PAY_COUNT_201802,PAY_COUNT_201803
0,6316050371,その他出金,28.0,28.0,32.0,27.0,26.0,28.0,28.0,29.0,32.0,29.0,27.0,29.0,27.0,31.0,33.0,28.0,28.0,27.0,26.0,28.0,28.0,30.0,28.0,30.0,27.0,32.0,34.0,27.0,30.0,29.0,32.0,26.0,29.0,27.0,27.0,27.0,26.0,30.0,31.0,28.0,28.0,26.0,29.0,28.0,29.0,29.0,27.0,30.0
1,6316050371,ガス料,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,6316050371,クレジット,6.0,8.0,6.0,7.0,9.0,6.0,6.0,7.0,7.0,7.0,6.0,6.0,7.0,7.0,6.0,7.0,10.0,6.0,6.0,6.0,8.0,7.0,7.0,5.0,5.0,6.0,5.0,6.0,7.0,6.0,4.0,5.0,7.0,5.0,6.0,6.0,8.0,9.0,7.0,8.0,10.0,7.0,8.0,9.0,11.0,10.0,9.0,9.0
3,6316050371,摘要なし,19.0,29.0,21.0,26.0,26.0,25.0,24.0,25.0,26.0,16.0,24.0,27.0,29.0,28.0,22.0,28.0,23.0,29.0,26.0,20.0,25.0,20.0,23.0,22.0,26.0,23.0,22.0,25.0,23.0,22.0,20.0,17.0,19.0,18.0,18.0,21.0,23.0,22.0,22.0,19.0,20.0,20.0,20.0,19.0,16.0,18.0,18.0,22.0
4,6316050371,群銀ＶＩＳＡ,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## 指定期間での分割横持ち化

In [71]:
# 入金金額
df_forplot_tmp = get_monthly_period(df_nyukin_tekiyo_amount_monthly, months, period_length, True)
df_nyukin_tekiyo_amount_monthly_periods_forplot = get_monthly_period_by_tekiyo(df_forplot_tmp)
df_nyukin_tekiyo_amount_monthly_periods_forplot = apply_suffix_tekiyo(df_nyukin_tekiyo_amount_monthly_periods_forplot, '入金_金額')

In [72]:
print(df_nyukin_tekiyo_amount_monthly_periods_forplot.shape)
df_nyukin_tekiyo_amount_monthly_periods_forplot.head()

(85788, 67)


,GODCUSTCD,YYYYMM,0_摘要なし_入金_金額,1_摘要なし_入金_金額,2_摘要なし_入金_金額,3_摘要なし_入金_金額,4_摘要なし_入金_金額,5_摘要なし_入金_金額,6_摘要なし_入金_金額,7_摘要なし_入金_金額,8_摘要なし_入金_金額,9_摘要なし_入金_金額,10_摘要なし_入金_金額,11_摘要なし_入金_金額,摘要なし_MEAN_入金_金額,0_当行株式配当金_入金_金額,1_当行株式配当金_入金_金額,2_当行株式配当金_入金_金額,3_当行株式配当金_入金_金額,4_当行株式配当金_入金_金額,5_当行株式配当金_入金_金額,6_当行株式配当金_入金_金額,7_当行株式配当金_入金_金額,8_当行株式配当金_入金_金額,9_当行株式配当金_入金_金額,10_当行株式配当金_入金_金額,11_当行株式配当金_入金_金額,当行株式配当金_MEAN_入金_金額,0_当行以外株式配当金_入金_金額,1_当行以外株式配当金_入金_金額,2_当行以外株式配当金_入金_金額,3_当行以外株式配当金_入金_金額,4_当行以外株式配当金_入金_金額,5_当行以外株式配当金_入金_金額,6_当行以外株式配当金_入金_金額,7_当行以外株式配当金_入金_金額,8_当行以外株式配当金_入金_金額,9_当行以外株式配当金_入金_金額,10_当行以外株式配当金_入金_金額,11_当行以外株式配当金_入金_金額,当行以外株式配当金_MEAN_入金_金額,0_その他入金_入金_金額,1_その他入金_入金_金額,2_その他入金_入金_金額,3_その他入金_入金_金額,4_その他入金_入金_金額,5_その他入金_入金_金額,6_その他入金_入金_金額,7_その他入金_入金_金額,8_その他入金_入金_金額,9_その他入金_入金_金額,10_その他入金_入金_金額,11_その他入金_入金_金額,その他入金_MEAN_入金_金額,0_給料_入金_金額,1_給料_入金_金額,2_給料_入金_金額,3_給料_入金_金額,4_給料_入金_金額,5_給料_入金_金額,6_給料_入金_金額,7_給料_入金_金額,8_給料_入金_金額,9_給料_入金_金額,10_給料_入金_金額,11_給料_入金_金額,給料_MEAN_入金_金額
0,6316050371,201503,1.609522e+08,1.442567e+08,4.650102e+07,1.488353e+08,1.104884e+08,1.342007e+08,1.766987e+08,1.877868e+08,1.478701e+08,1.498025e+08,4.646600e+07,1.291094e+08,1.319140e+08,0.0,0.0,8469.0,0.0,0.0,0.0,0.0,0.0,7622.0,0.0,0.0,0.0,1340.916667,0.0,0.0,0.0,0.0,1520.0,0.0,0.0,0.0,0.0,0.0,1286.0,0.0,233.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6316050390,201503,1.299387e+08,3.305081e+08,3.953656e+08,1.303706e+08,1.915690e+08,2.324051e+08,1.073291e+08,1.414672e+08,2.391273e+08,3.369461e+08,5.200510e+08,1.904705e+08,2.454624e+08,0.0,0.0,419191.0,0.0,0.0,0.0,0.0,0.0,377272.0,0.0,0.0,0.0,66371.916667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6316050420,201503,2.006548e+08,1.983726e+08,1.714708e+08,1.591328e+08,1.425629e+08,1.383968e+08,1.447669e+08,1.341446e+08,1.993109e+08,2.306400e+08,2.421135e+08,2.338046e+08,1.829476e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,54657.0,0.0,0.0,0.0,0.0,0.0,48305.0,0.0,0.0,0.0,8580.166667,31078099.0,54556405.0,17243011.0,37634549.0,22243700.0,14987949.0,33247368.0,11267095.0,32217710.0,43277693.0,29875253.0,32038322.0,2.997226e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6316050622,201503,1.968210e+09,1.579108e+09,1.918852e+09,1.702694e+09,1.453711e+09,1.664398e+09,1.549506e+09,2.096703e+09,1.670889e+09,1.490563e+09,1.968236e+09,1.893088e+09,1.746330e+09,0.0,0.0,119495.0,0.0,0.0,0.0,0.0,0.0,107546.0,0.0,0.0,0.0,18920.083333,0.0,0.0,997590.0,0.0,0.0,0.0,0.0,0.0,965409.0,0.0,0.0,0.0,163583.250000,42357654.0,70663540.0,52727836.0,37632015.0,50584059.0,41877201.0,9135140.0,73671536.0,68545665.0,66588026.0,127136561.0,63027119.0,5.866220e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6316050673,201503,9.700000e+05,9.400000e+05,9.350000e+05,9.500000e+05,9.300060e+05,9.500000e+05,9.500000e+05,9.000000e+05,9.400000e+05,9.200000e+05,9.000150e+05,9.400000e+05,9.354184e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# 入金件数
df_forplot_tmp = get_monthly_period(df_nyukin_tekiyo_count_monthly, months, period_length, True)
df_nyukin_tekiyo_count_monthly_periods_forplot = get_monthly_period_by_tekiyo(df_forplot_tmp)
df_nyukin_tekiyo_count_monthly_periods_forplot = apply_suffix_tekiyo(df_nyukin_tekiyo_count_monthly_periods_forplot, '入金_件数')

In [74]:
print(df_nyukin_tekiyo_count_monthly_periods_forplot.shape)
df_nyukin_tekiyo_count_monthly_periods_forplot.head()

(85788, 67)


,GODCUSTCD,YYYYMM,0_摘要なし_入金_件数,1_摘要なし_入金_件数,2_摘要なし_入金_件数,3_摘要なし_入金_件数,4_摘要なし_入金_件数,5_摘要なし_入金_件数,6_摘要なし_入金_件数,7_摘要なし_入金_件数,8_摘要なし_入金_件数,9_摘要なし_入金_件数,10_摘要なし_入金_件数,11_摘要なし_入金_件数,摘要なし_MEAN_入金_件数,0_当行株式配当金_入金_件数,1_当行株式配当金_入金_件数,2_当行株式配当金_入金_件数,3_当行株式配当金_入金_件数,4_当行株式配当金_入金_件数,5_当行株式配当金_入金_件数,6_当行株式配当金_入金_件数,7_当行株式配当金_入金_件数,8_当行株式配当金_入金_件数,9_当行株式配当金_入金_件数,10_当行株式配当金_入金_件数,11_当行株式配当金_入金_件数,当行株式配当金_MEAN_入金_件数,0_当行以外株式配当金_入金_件数,1_当行以外株式配当金_入金_件数,2_当行以外株式配当金_入金_件数,3_当行以外株式配当金_入金_件数,4_当行以外株式配当金_入金_件数,5_当行以外株式配当金_入金_件数,6_当行以外株式配当金_入金_件数,7_当行以外株式配当金_入金_件数,8_当行以外株式配当金_入金_件数,9_当行以外株式配当金_入金_件数,10_当行以外株式配当金_入金_件数,11_当行以外株式配当金_入金_件数,当行以外株式配当金_MEAN_入金_件数,0_その他入金_入金_件数,1_その他入金_入金_件数,2_その他入金_入金_件数,3_その他入金_入金_件数,4_その他入金_入金_件数,5_その他入金_入金_件数,6_その他入金_入金_件数,7_その他入金_入金_件数,8_その他入金_入金_件数,9_その他入金_入金_件数,10_その他入金_入金_件数,11_その他入金_入金_件数,その他入金_MEAN_入金_件数,0_給料_入金_件数,1_給料_入金_件数,2_給料_入金_件数,3_給料_入金_件数,4_給料_入金_件数,5_給料_入金_件数,6_給料_入金_件数,7_給料_入金_件数,8_給料_入金_件数,9_給料_入金_件数,10_給料_入金_件数,11_給料_入金_件数,給料_MEAN_入金_件数
0,6316050371,201503,101.0,63.0,59.0,57.0,78.0,80.0,72.0,70.0,60.0,53.0,66.0,80.0,69.916667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6316050390,201503,58.0,67.0,79.0,62.0,60.0,72.0,65.0,59.0,69.0,53.0,63.0,64.0,64.250000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6316050420,201503,322.0,299.0,328.0,302.0,285.0,315.0,305.0,277.0,348.0,299.0,305.0,346.0,310.916667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.250000,18.0,20.0,17.0,20.0,14.0,13.0,16.0,13.0,18.0,13.0,15.0,17.0,16.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6316050622,201503,878.0,781.0,880.0,830.0,793.0,837.0,850.0,786.0,934.0,898.0,859.0,920.0,853.833333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.166667,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.250000,36.0,36.0,42.0,35.0,34.0,36.0,33.0,36.0,36.0,38.0,37.0,37.0,36.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6316050673,201503,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
# 出金金額
df_forplot_tmp = get_monthly_period(df_shukkin_tekiyo_amount_monthly, months, period_length, True)
df_shukkin_tekiyo_amount_monthly_periods_forplot = get_monthly_period_by_tekiyo(df_forplot_tmp)
df_shukkin_tekiyo_amount_monthly_periods_forplot = apply_suffix_tekiyo(df_shukkin_tekiyo_amount_monthly_periods_forplot, '出金_金額')

In [76]:
print(df_shukkin_tekiyo_amount_monthly_periods_forplot.shape)
df_shukkin_tekiyo_amount_monthly_periods_forplot.head()

(86112, 171)


,GODCUSTCD,YYYYMM,0_その他出金_出金_金額,1_その他出金_出金_金額,2_その他出金_出金_金額,3_その他出金_出金_金額,4_その他出金_出金_金額,5_その他出金_出金_金額,6_その他出金_出金_金額,7_その他出金_出金_金額,8_その他出金_出金_金額,9_その他出金_出金_金額,10_その他出金_出金_金額,11_その他出金_出金_金額,その他出金_MEAN_出金_金額,0_ガス料_出金_金額,1_ガス料_出金_金額,2_ガス料_出金_金額,3_ガス料_出金_金額,4_ガス料_出金_金額,5_ガス料_出金_金額,6_ガス料_出金_金額,7_ガス料_出金_金額,8_ガス料_出金_金額,9_ガス料_出金_金額,10_ガス料_出金_金額,11_ガス料_出金_金額,ガス料_MEAN_出金_金額,0_クレジット_出金_金額,1_クレジット_出金_金額,2_クレジット_出金_金額,3_クレジット_出金_金額,4_クレジット_出金_金額,5_クレジット_出金_金額,6_クレジット_出金_金額,7_クレジット_出金_金額,8_クレジット_出金_金額,9_クレジット_出金_金額,10_クレジット_出金_金額,11_クレジット_出金_金額,クレジット_MEAN_出金_金額,0_摘要なし_出金_金額,1_摘要なし_出金_金額,2_摘要なし_出金_金額,3_摘要なし_出金_金額,4_摘要なし_出金_金額,5_摘要なし_出金_金額,6_摘要なし_出金_金額,7_摘要なし_出金_金額,8_摘要なし_出金_金額,9_摘要なし_出金_金額,10_摘要なし_出金_金額,11_摘要なし_出金_金額,摘要なし_MEAN_出金_金額,0_群銀ＶＩＳＡ_出金_金額,1_群銀ＶＩＳＡ_出金_金額,2_群銀ＶＩＳＡ_出金_金額,3_群銀ＶＩＳＡ_出金_金額,4_群銀ＶＩＳＡ_出金_金額,5_群銀ＶＩＳＡ_出金_金額,6_群銀ＶＩＳＡ_出金_金額,7_群銀ＶＩＳＡ_出金_金額,8_群銀ＶＩＳＡ_出金_金額,9_群銀ＶＩＳＡ_出金_金額,10_群銀ＶＩＳＡ_出金_金額,11_群銀ＶＩＳＡ_出金_金額,群銀ＶＩＳＡ_MEAN_出金_金額,0_電気料_出金_金額,1_電気料_出金_金額,2_電気料_出金_金額,3_電気料_出金_金額,4_電気料_出金_金額,5_電気料_出金_金額,6_電気料_出金_金額,7_電気料_出金_金額,8_電気料_出金_金額,9_電気料_出金_金額,10_電気料_出金_金額,11_電気料_出金_金額,電気料_MEAN_出金_金額,0_電話料_出金_金額,1_電話料_出金_金額,2_電話料_出金_金額,3_電話料_出金_金額,4_電話料_出金_金額,5_電話料_出金_金額,6_電話料_出金_金額,7_電話料_出金_金額,8_電話料_出金_金額,9_電話料_出金_金額,10_電話料_出金_金額,11_電話料_出金_金額,電話料_MEAN_出金_金額,0_国保・社保_出金_金額,1_国保・社保_出金_金額,2_国保・社保_出金_金額,3_国保・社保_出金_金額,4_国保・社保_出金_金額,5_国保・社保_出金_金額,6_国保・社保_出金_金額,7_国保・社保_出金_金額,8_国保・社保_出金_金額,9_国保・社保_出金_金額,10_国保・社保_出金_金額,11_国保・社保_出金_金額,国保・社保_MEAN_出金_金額,0_国税・地方税_出金_金額,1_国税・地方税_出金_金額,2_国税・地方税_出金_金額,3_国税・地方税_出金_金額,4_国税・地方税_出金_金額,5_国税・地方税_出金_金額,6_国税・地方税_出金_金額,7_国税・地方税_出金_金額,8_国税・地方税_出金_金額,9_国税・地方税_出金_金額,10_国税・地方税_出金_金額,11_国税・地方税_出金_金額,国税・地方税_MEAN_出金_金額,0_水道料_出金_金額,1_水道料_出金_金額,2_水道料_出金_金額,3_水道料_出金_金額,4_水道料_出金_金額,5_水道料_出金_金額,6_水道料_出金_金額,7_水道料_出金_金額,8_水道料_出金_金額,9_水道料_出金_金額,10_水道料_出金_金額,11_水道料_出金_金額,水道料_MEAN_出金_金額,0_貸金庫_出金_金額,1_貸金庫_出金_金額,2_貸金庫_出金_金額,3_貸金庫_出金_金額,4_貸金庫_出金_金額,5_貸金庫_出金_金額,6_貸金庫_出金_金額,7_貸金庫_出金_金額,8_貸金庫_出金_金額,9_貸金庫_出金_金額,10_貸金庫_出金_金額,11_貸金庫_出金_金額,貸金庫_MEAN_出金_金額,0_ＮＨＫ受信料_出金_金額,1_ＮＨＫ受信料_出金_金額,2_ＮＨＫ受信料_出金_金額,3_ＮＨＫ受信料_出金_金額,4_ＮＨＫ受信料_出金_金額,5_ＮＨＫ受信料_出金_金額,6_ＮＨＫ受信料_出金_金額,7_ＮＨＫ受信料_出金_金額,8_ＮＨＫ受信料_出金_金額,9_ＮＨＫ受信料_出金_金額,10_ＮＨＫ受信料_出金_金額,11_ＮＨＫ受信料_出金_金額,ＮＨＫ受信料_MEAN_出金_金額,0_ぐんぎんＪＣＢ_出金_金額,1_ぐんぎんＪＣＢ_出金_金額,2_ぐんぎんＪＣＢ_出金_金額,3_ぐんぎんＪＣＢ_出金_金額,4_ぐんぎんＪＣＢ_出金_金額,5_ぐんぎんＪＣＢ_出金_金額,6_ぐんぎんＪＣＢ_出金_金額,7_ぐんぎんＪＣＢ_出金_金額,8_ぐんぎんＪＣＢ_出金_金額,9_ぐんぎんＪＣＢ_出金_金額,10_ぐんぎんＪＣＢ_出金_金額,11_ぐんぎんＪＣＢ_出金_金額,ぐんぎんＪＣＢ_MEAN_出金_金額
0,6316050371,201503,13242874.0,11278738.0,24145073.0,12566812.0,11911535.0,15302422.0,13616765.0,12326121.0,21613660.0,12756140.0,13655479.0,15545042.0,1.483006e+07,1575.0,1575.0,1620.0,1620.0,1620.0,1620.0,1620.0,1620.0,1620.0,1620.0,2882.0,2120.0,1759.333333,174995.0,324802.0,127925.0,143466.0,258432.0,200285.0,127925.0,143466.0,168817.0,174395.0,164955.0,132275.0,1.784782e+05,9.805186e+07,1.458532e+08,8.951288e+07,1.308874e+08,7.856163e+07,1.330993e+08,1.778946e+08,1.126276e+08,5.707593e+07,2.323142e+08,5.433677e+07,9.698864e+07,1.172670e+08,239677.0,331030.0,313340.0,339949.0,348417.0,347804.0,359159.0,568843.0,295258.0,341081.0,276615.0,416945.0,348176.5,4672.0,4771.0,4300.0,4704.0,4264.0,4745.0,4990.0,4673.0,3479.0,4338.0,4052.0,4290.0,4.439833e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6316050390,201503,95691593.0,118598206.0,95336973.0,170741091.0,121251141.0,82684664.0,107251132.0,123302050.0,132462053.0,81506376.0,131099044.0,155007887.0,1.179110e+08,1054.0,1091.0,1091.0,824.0,957.0,823.0,955.0,1087.0,1087.0,1088.0,1090.0,959.0,1008.833333,89631.0,144529.0,161417.0,169791.0,184590.0,137831.0,176556.0,227694.0,159306.0,236242.0,301041.0,163268.0,1.793247e+05,9.640058e+06,2.125436e+08,2.476806e+08,2.490613e+06,1.768862e+07,1.267351e+08,4.616930e+07,1

In [77]:
# 出金件数
df_forplot_tmp = get_monthly_period(df_shukkin_tekiyo_count_monthly, months, period_length, True)
df_shukkin_tekiyo_count_monthly_periods_forplot = get_monthly_period_by_tekiyo(df_forplot_tmp)
df_shukkin_tekiyo_count_monthly_periods_forplot = apply_suffix_tekiyo(df_shukkin_tekiyo_count_monthly_periods_forplot, '出金_件数')

In [78]:
print(df_shukkin_tekiyo_count_monthly_periods_forplot.shape)
df_shukkin_tekiyo_count_monthly_periods_forplot.head()

(86112, 171)


,GODCUSTCD,YYYYMM,0_その他出金_出金_件数,1_その他出金_出金_件数,2_その他出金_出金_件数,3_その他出金_出金_件数,4_その他出金_出金_件数,5_その他出金_出金_件数,6_その他出金_出金_件数,7_その他出金_出金_件数,8_その他出金_出金_件数,9_その他出金_出金_件数,10_その他出金_出金_件数,11_その他出金_出金_件数,その他出金_MEAN_出金_件数,0_ガス料_出金_件数,1_ガス料_出金_件数,2_ガス料_出金_件数,3_ガス料_出金_件数,4_ガス料_出金_件数,5_ガス料_出金_件数,6_ガス料_出金_件数,7_ガス料_出金_件数,8_ガス料_出金_件数,9_ガス料_出金_件数,10_ガス料_出金_件数,11_ガス料_出金_件数,ガス料_MEAN_出金_件数,0_クレジット_出金_件数,1_クレジット_出金_件数,2_クレジット_出金_件数,3_クレジット_出金_件数,4_クレジット_出金_件数,5_クレジット_出金_件数,6_クレジット_出金_件数,7_クレジット_出金_件数,8_クレジット_出金_件数,9_クレジット_出金_件数,10_クレジット_出金_件数,11_クレジット_出金_件数,クレジット_MEAN_出金_件数,0_摘要なし_出金_件数,1_摘要なし_出金_件数,2_摘要なし_出金_件数,3_摘要なし_出金_件数,4_摘要なし_出金_件数,5_摘要なし_出金_件数,6_摘要なし_出金_件数,7_摘要なし_出金_件数,8_摘要なし_出金_件数,9_摘要なし_出金_件数,10_摘要なし_出金_件数,11_摘要なし_出金_件数,摘要なし_MEAN_出金_件数,0_群銀ＶＩＳＡ_出金_件数,1_群銀ＶＩＳＡ_出金_件数,2_群銀ＶＩＳＡ_出金_件数,3_群銀ＶＩＳＡ_出金_件数,4_群銀ＶＩＳＡ_出金_件数,5_群銀ＶＩＳＡ_出金_件数,6_群銀ＶＩＳＡ_出金_件数,7_群銀ＶＩＳＡ_出金_件数,8_群銀ＶＩＳＡ_出金_件数,9_群銀ＶＩＳＡ_出金_件数,10_群銀ＶＩＳＡ_出金_件数,11_群銀ＶＩＳＡ_出金_件数,群銀ＶＩＳＡ_MEAN_出金_件数,0_電気料_出金_件数,1_電気料_出金_件数,2_電気料_出金_件数,3_電気料_出金_件数,4_電気料_出金_件数,5_電気料_出金_件数,6_電気料_出金_件数,7_電気料_出金_件数,8_電気料_出金_件数,9_電気料_出金_件数,10_電気料_出金_件数,11_電気料_出金_件数,電気料_MEAN_出金_件数,0_電話料_出金_件数,1_電話料_出金_件数,2_電話料_出金_件数,3_電話料_出金_件数,4_電話料_出金_件数,5_電話料_出金_件数,6_電話料_出金_件数,7_電話料_出金_件数,8_電話料_出金_件数,9_電話料_出金_件数,10_電話料_出金_件数,11_電話料_出金_件数,電話料_MEAN_出金_件数,0_国保・社保_出金_件数,1_国保・社保_出金_件数,2_国保・社保_出金_件数,3_国保・社保_出金_件数,4_国保・社保_出金_件数,5_国保・社保_出金_件数,6_国保・社保_出金_件数,7_国保・社保_出金_件数,8_国保・社保_出金_件数,9_国保・社保_出金_件数,10_国保・社保_出金_件数,11_国保・社保_出金_件数,国保・社保_MEAN_出金_件数,0_国税・地方税_出金_件数,1_国税・地方税_出金_件数,2_国税・地方税_出金_件数,3_国税・地方税_出金_件数,4_国税・地方税_出金_件数,5_国税・地方税_出金_件数,6_国税・地方税_出金_件数,7_国税・地方税_出金_件数,8_国税・地方税_出金_件数,9_国税・地方税_出金_件数,10_国税・地方税_出金_件数,11_国税・地方税_出金_件数,国税・地方税_MEAN_出金_件数,0_水道料_出金_件数,1_水道料_出金_件数,2_水道料_出金_件数,3_水道料_出金_件数,4_水道料_出金_件数,5_水道料_出金_件数,6_水道料_出金_件数,7_水道料_出金_件数,8_水道料_出金_件数,9_水道料_出金_件数,10_水道料_出金_件数,11_水道料_出金_件数,水道料_MEAN_出金_件数,0_貸金庫_出金_件数,1_貸金庫_出金_件数,2_貸金庫_出金_件数,3_貸金庫_出金_件数,4_貸金庫_出金_件数,5_貸金庫_出金_件数,6_貸金庫_出金_件数,7_貸金庫_出金_件数,8_貸金庫_出金_件数,9_貸金庫_出金_件数,10_貸金庫_出金_件数,11_貸金庫_出金_件数,貸金庫_MEAN_出金_件数,0_ＮＨＫ受信料_出金_件数,1_ＮＨＫ受信料_出金_件数,2_ＮＨＫ受信料_出金_件数,3_ＮＨＫ受信料_出金_件数,4_ＮＨＫ受信料_出金_件数,5_ＮＨＫ受信料_出金_件数,6_ＮＨＫ受信料_出金_件数,7_ＮＨＫ受信料_出金_件数,8_ＮＨＫ受信料_出金_件数,9_ＮＨＫ受信料_出金_件数,10_ＮＨＫ受信料_出金_件数,11_ＮＨＫ受信料_出金_件数,ＮＨＫ受信料_MEAN_出金_件数,0_ぐんぎんＪＣＢ_出金_件数,1_ぐんぎんＪＣＢ_出金_件数,2_ぐんぎんＪＣＢ_出金_件数,3_ぐんぎんＪＣＢ_出金_件数,4_ぐんぎんＪＣＢ_出金_件数,5_ぐんぎんＪＣＢ_出金_件数,6_ぐんぎんＪＣＢ_出金_件数,7_ぐんぎんＪＣＢ_出金_件数,8_ぐんぎんＪＣＢ_出金_件数,9_ぐんぎんＪＣＢ_出金_件数,10_ぐんぎんＪＣＢ_出金_件数,11_ぐんぎんＪＣＢ_出金_件数,ぐんぎんＪＣＢ_MEAN_出金_件数
0,6316050371,201503,28.0,28.0,32.0,27.0,26.0,28.0,28.0,29.0,32.0,29.0,27.0,29.0,28.583333,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,6.0,8.0,6.0,7.0,9.0,6.0,6.0,7.0,7.0,7.0,6.0,6.0,6.750000,19.0,29.0,21.0,26.0,26.0,25.0,24.0,25.0,26.0,16.0,24.0,27.0,24.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,3.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6316050390,201503,19.0,20.0,17.0,20.0,13.0,18.0,18.0,17.0,17.0,18.0,17.0,17.0,17.583333,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,2.250000,8.0,17.0,15.0,7.0,12.0,11.0,10.0,8.0,8.0,14.0,19.0,12.0,11.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.000000,4.0,0.0,8.0,4.0,0.0,8.0,4.0,0.0,4.0,4.0,4.0,8.0,4.000000,1.0,0.0,2.0,1.0,0.0,3.0,1.0,1.0,1.0,1.0,2.0,2.0,1.25,1.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.500000,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,1.500000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6316050420,201503,16.0,18.0,19.0,22.0,16.0,17.0,16.0,17.0,19.0,17.0,15.0,16.0,17.333333,0.0,0.0

## 残高データの取得

### 月末残高（普通）

In [79]:
df_zandaka_last_futu = godcustcd_preprocessing(pd.read_csv(PATH_INPUT+'zandaka_last_futu.csv', index_col=0))
df_zandaka_last_futu = df_zandaka_last_futu[['GODCUSTCD','YYYYMM','ZANDAKA']]
df_zandaka_last_futu.rename(columns={'ZANDAKA':'ZANDAKA_LAST_FUTU'}, inplace=True)

In [80]:
print(df_zandaka_last_futu.shape)
df_zandaka_last_futu.head()

(106270, 3)


,GODCUSTCD,YYYYMM,ZANDAKA_LAST_FUTU
0,6316050312,201404,86053387
1,6316050312,201405,61423387
2,6316050312,201406,61580128
3,6316050312,201407,61204967
4,6316050312,201408,101768048


### 月末残高（当座）

In [81]:
df_zandaka_last_toza = godcustcd_preprocessing(pd.read_csv(PATH_INPUT+'zandaka_last_toza.csv', index_col=0))
df_zandaka_last_toza = df_zandaka_last_toza[['GODCUSTCD','YYYYMM','ZANDAKA']]
df_zandaka_last_toza.rename(columns={'ZANDAKA':'ZANDAKA_LAST_TOZA'}, inplace=True)

In [82]:
print(df_zandaka_last_toza.shape)
df_zandaka_last_toza.head()

(37431, 3)


,GODCUSTCD,YYYYMM,ZANDAKA_LAST_TOZA
0,6316050312,201404,128444458
1,6316050312,201405,119160459
2,6316050312,201406,197201320
3,6316050312,201407,75066719
4,6316050312,201408,65203443


### 月末残高（普通・当座）

In [83]:
df_zandaka_last = pd.merge(df_zandaka_last_futu, df_zandaka_last_toza, on=['GODCUSTCD','YYYYMM'], how='outer').fillna(0)
df_zandaka_last['ZANDAKA_LAST'] = df_zandaka_last['ZANDAKA_LAST_FUTU'] + df_zandaka_last['ZANDAKA_LAST_TOZA']
df_zandaka_last.drop(['ZANDAKA_LAST_FUTU','ZANDAKA_LAST_TOZA'], axis=1, inplace=True)

In [84]:
print(df_zandaka_last.shape)
df_zandaka_last.head()

(120648, 3)


,GODCUSTCD,YYYYMM,ZANDAKA_LAST
0,6316050312,201404,214497845.0
1,6316050312,201405,180583846.0
2,6316050312,201406,258781448.0
3,6316050312,201407,136271686.0
4,6316050312,201408,166971491.0


## 月末残高データの横持ち化

In [85]:
CUST_KEY = ['GODCUSTCD'] 

In [86]:
# 　月末残高（普通・当座）横持ち
df_cust_master = pd.DataFrame(df_zandaka_last[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_zandaka_last_monthly  = get_zandaka_monthly_yokomochi(
    df_cust_master.copy(), df_zandaka_last, 'YYYYMM', 'ZANDAKA_LAST')

In [87]:
print(df_zandaka_last_monthly.shape)
df_zandaka_last_monthly.head()

(2889, 49)


,GODCUSTCD,ZANDAKA_LAST_201404,ZANDAKA_LAST_201405,ZANDAKA_LAST_201406,ZANDAKA_LAST_201407,ZANDAKA_LAST_201408,ZANDAKA_LAST_201409,ZANDAKA_LAST_201410,ZANDAKA_LAST_201411,ZANDAKA_LAST_201412,ZANDAKA_LAST_201501,ZANDAKA_LAST_201502,ZANDAKA_LAST_201503,ZANDAKA_LAST_201504,ZANDAKA_LAST_201505,ZANDAKA_LAST_201506,ZANDAKA_LAST_201507,ZANDAKA_LAST_201508,ZANDAKA_LAST_201509,ZANDAKA_LAST_201510,ZANDAKA_LAST_201511,ZANDAKA_LAST_201512,ZANDAKA_LAST_201601,ZANDAKA_LAST_201602,ZANDAKA_LAST_201603,ZANDAKA_LAST_201604,ZANDAKA_LAST_201605,ZANDAKA_LAST_201606,ZANDAKA_LAST_201607,ZANDAKA_LAST_201608,ZANDAKA_LAST_201609,ZANDAKA_LAST_201610,ZANDAKA_LAST_201611,ZANDAKA_LAST_201612,ZANDAKA_LAST_201701,ZANDAKA_LAST_201702,ZANDAKA_LAST_201703,ZANDAKA_LAST_201704,ZANDAKA_LAST_201705,ZANDAKA_LAST_201706,ZANDAKA_LAST_201707,ZANDAKA_LAST_201708,ZANDAKA_LAST_201709,ZANDAKA_LAST_201710,ZANDAKA_LAST_201711,ZANDAKA_LAST_201712,ZANDAKA_LAST_201801,ZANDAKA_LAST_201802,ZANDAKA_LAST_201803
0,6316050312,214497845.0,180583846.0,258781448.0,136271686.0,166971491.0,245896200.0,333885505.0,262205606.0,279639033.0,562457872.0,381366002.0,473741486.0,218889090.0,151197333.0,62327594.0,195568317.0,316750415.0,239589292.0,289341173.0,217774560.0,453862749.0,214548611.0,202193933.0,199120786.0,398175414.0,254338215.0,219038367.0,360071799.0,209059250.0,136229342.0,235538057.0,172015278.0,259455553.0,170796792.0,181204166.0,186220866.0,295178516.0,108294828.0,273800133.0,287015091.0,183177930.0,212473313.0,165511324.0,229350234.0,285067828.0,213469530.0,168165026.0,179764713.0
1,6316050371,96661035.0,83123577.0,15527930.0,20419349.0,39823393.0,25067866.0,9761479.0,71875996.0,140594971.0,44805615.0,22832144.0,38852233.0,33127313.0,51015974.0,30874775.0,52167296.0,30842104.0,62656992.0,175516461.0,125816211.0,122182165.0,114547792.0,118858091.0,183843277.0,146525821.0,285275357.0,137334479.0,210070249.0,83439159.0,5939231.0,14773803.0,82969676.0,12378001.0,32110831.0,44048761.0,64107668.0,41849122.0,27412485.0,56802355.0,87996082.0,23031605.0,179756436.0,13116863.0,29858768.0,33408207.0,28098473.0,24380940.0,130976677.0
2,6316050390,98331211.0,96057754.0,131224390.0,77097005.0,127660180.0,124779822.0,69034347.0,66706756.0,76106262.0,156095537.0,136891401.0,111597092.0,112281035.0,110357377.0,102159373.0,125874722.0,120592458.0,158982308.0,99241417.0,93913845.0,111218732.0,71825487.0,49563693.0,56991708.0,78871920.0,135432439.0,127520090.0,93177170.0,362878879.0,217062405.0,112988682.0,114152452.0,128125498.0,319882817.0,109282092.0,103737665.0,84813771.0,74608438.0,58924074.0,430729250.0,125095867.0,108263386.0,70559389.0,85450364.0,153171164.0,148871480.0,186789185.0,131452439.0
3,6316050420,307852228.0,332209104.0,312887224.0,312948417.0,313061457.0,280519285.0,300487165.0,292940748.0,320613497.0,345065152.0,360806778.0,368172911.0,371275093.0,366701530.0,346485179.0,355164504.0,332146387.0,340168080.0,357022651.0,306990410.0,325389263.0,335284828.0,339982175.0,355437247.0,360811226.0,372130480.0,384359869.0,363762242.0,353130301.0,363675198.0,369793901.0,386400008.0,402319116.0,399595658.0,393728155.0,424868707.0,434541474.0,437852002.0,444508079.0,453325058.0,426574576.0,433780371.0,414422223.0,405681164.0,413532985.0,489464389.0,391576755.0,131790868.0
4,6316050622,381101671.0,310390466.0,231805432.0,186006288.0,172915026.0,311031998.0,255089466.0,255291955.0,258303654.0,323289283.0,296469160.0,735120019.0,243349878.0,202265460.0,242411445.0,305033344.0,214585818.0,439907790.0,287858773.0,236509589.0,216556674.0,340407050.0,228424571.0,310287619.0,319275617.0,384685023.0,235792615.0,484555180.0,206911093.0,281139949.0,139358307.0,110593222.0,298184201.0,178627505.0,149753271.0,325578183.0,259353827.0,199984976.0,219653001.0,170778064.0,151722295.0,254164129.0,134872661.0,149258980.0,418013652.0,451085487.0,457967856.0,574105549.0


In [88]:
# 　月末残高（普通）横持ち
df_cust_master = pd.DataFrame(df_zandaka_last_futu[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_zandaka_last_futu_monthly  = get_zandaka_monthly_yokomochi(
    df_cust_master.copy(), df_zandaka_last_futu, 'YYYYMM', 'ZANDAKA_LAST_FUTU')

In [89]:
print(df_zandaka_last_futu_monthly.shape)
df_zandaka_last_futu_monthly.head()

(2626, 49)


,GODCUSTCD,ZANDAKA_LAST_FUTU_201404,ZANDAKA_LAST_FUTU_201405,ZANDAKA_LAST_FUTU_201406,ZANDAKA_LAST_FUTU_201407,ZANDAKA_LAST_FUTU_201408,ZANDAKA_LAST_FUTU_201409,ZANDAKA_LAST_FUTU_201410,ZANDAKA_LAST_FUTU_201411,ZANDAKA_LAST_FUTU_201412,ZANDAKA_LAST_FUTU_201501,ZANDAKA_LAST_FUTU_201502,ZANDAKA_LAST_FUTU_201503,ZANDAKA_LAST_FUTU_201504,ZANDAKA_LAST_FUTU_201505,ZANDAKA_LAST_FUTU_201506,ZANDAKA_LAST_FUTU_201507,ZANDAKA_LAST_FUTU_201508,ZANDAKA_LAST_FUTU_201509,ZANDAKA_LAST_FUTU_201510,ZANDAKA_LAST_FUTU_201511,ZANDAKA_LAST_FUTU_201512,ZANDAKA_LAST_FUTU_201601,ZANDAKA_LAST_FUTU_201602,ZANDAKA_LAST_FUTU_201603,ZANDAKA_LAST_FUTU_201604,ZANDAKA_LAST_FUTU_201605,ZANDAKA_LAST_FUTU_201606,ZANDAKA_LAST_FUTU_201607,ZANDAKA_LAST_FUTU_201608,ZANDAKA_LAST_FUTU_201609,ZANDAKA_LAST_FUTU_201610,ZANDAKA_LAST_FUTU_201611,ZANDAKA_LAST_FUTU_201612,ZANDAKA_LAST_FUTU_201701,ZANDAKA_LAST_FUTU_201702,ZANDAKA_LAST_FUTU_201703,ZANDAKA_LAST_FUTU_201704,ZANDAKA_LAST_FUTU_201705,ZANDAKA_LAST_FUTU_201706,ZANDAKA_LAST_FUTU_201707,ZANDAKA_LAST_FUTU_201708,ZANDAKA_LAST_FUTU_201709,ZANDAKA_LAST_FUTU_201710,ZANDAKA_LAST_FUTU_201711,ZANDAKA_LAST_FUTU_201712,ZANDAKA_LAST_FUTU_201801,ZANDAKA_LAST_FUTU_201802,ZANDAKA_LAST_FUTU_201803
0,6316050312,86053387.0,61423387.0,61580128.0,61204967.0,101768048.0,87988048.0,112948048.0,100830254.0,81024718.0,67200718.0,60290868.0,44010868.0,35505238.0,22625238.0,39625238.0,53925238.0,48928780.0,63428780.0,116228780.0,132728780.0,99668780.0,72548780.0,55386366.0,41086366.0,18463166.0,13943166.0,79243166.0,71050904.0,80741527.0,81054421.0,106349977.0,114149977.0,74109977.0,74109977.0,32440361.0,32670361.0,650361.0,650361.0,33670361.0,110470361.0,103990507.0,91880507.0,53700507.0,82000507.0,82000507.0,46700507.0,26570841.0,24761781.0
1,6316050371,99335.0,98255.0,97175.0,96095.0,96103.0,96103.0,96103.0,96103.0,96103.0,96103.0,96111.0,96111.0,96111.0,96111.0,96111.0,96111.0,96119.0,96119.0,95310199.0,95310199.0,95310199.0,95310199.0,95315288.0,95315288.0,95315288.0,95315288.0,103288.0,103288.0,104307.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6316050390,5463212.0,14206366.0,7756351.0,6509835.0,2288342.0,42788106.0,3847189.0,2016279.0,3571873.0,2694901.0,14663664.0,11924707.0,9052602.0,15996092.0,9063144.0,6581334.0,13567048.0,11219699.0,13979197.0,21135461.0,9283195.0,7689423.0,3625440.0,2884651.0,20437256.0,22440470.0,16559856.0,14444278.0,8938776.0,6006359.0,3947336.0,3567037.0,8411182.0,6050216.0,5267032.0,6204615.0,3513839.0,9667792.0,6248886.0,2904638.0,10107163.0,6400536.0,13784320.0,9436664.0,6599198.0,25893026.0,8858486.0,6213187.0
3,6316050420,54514726.0,48860572.0,43915959.0,45745064.0,47808327.0,37524393.0,45654458.0,31117546.0,79011038.0,89712805.0,80967145.0,82513792.0,74912514.0,64141323.0,54242683.0,66002226.0,58600897.0,74354204.0,51041119.0,39966671.0,64272305.0,64713845.0,51523968.0,51233515.0,24680666.0,43508505.0,36463216.0,67479012.0,41454618.0,40080778.0,48092342.0,60527706.0,45810333.0,46260960.0,33419740.0,45543254.0,31394855.0,45306723.0,40938701.0,43569327.0,41296827.0,31478115.0,37924595.0,43889268.0,60721312.0,86279375.0,75970141.0,45512845.0
4,6316050622,39224716.0,24658944.0,50015179.0,35105003.0,22013623.0,32084020.0,49411097.0,27950616.0,22407991.0,23127138.0,23225308.0,35437751.0,39290570.0,25892458.0,63294522.0,39217521.0,39148250.0,27440734.0,24149011.0,45868535.0,23376666.0,23184760.0,35506418.0,37944310.0,23665653.0,63411424.0,62099349.0,27769608.0,32976034.0,35567293.0,29737292.0,40098581.0,23669303.0,34026123.0,35663141.0,36864314.0,25522561.0,36235428.0,58680597.0,38948319.0,35145592.0,24812478.0,17176651.0,37438153.0,26306268.0,69793834.0,71630058.0,25055281.0


In [90]:
# 　月末残高（当座）横持ち
df_cust_master = pd.DataFrame(df_zandaka_last_toza[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_zandaka_last_toza_monthly  = get_zandaka_monthly_yokomochi(
    df_cust_master.copy(), df_zandaka_last_toza, 'YYYYMM', 'ZANDAKA_LAST_TOZA')

In [91]:
print(df_zandaka_last_toza_monthly.shape)
df_zandaka_last_toza_monthly.head()

(842, 49)


,GODCUSTCD,ZANDAKA_LAST_TOZA_201404,ZANDAKA_LAST_TOZA_201405,ZANDAKA_LAST_TOZA_201406,ZANDAKA_LAST_TOZA_201407,ZANDAKA_LAST_TOZA_201408,ZANDAKA_LAST_TOZA_201409,ZANDAKA_LAST_TOZA_201410,ZANDAKA_LAST_TOZA_201411,ZANDAKA_LAST_TOZA_201412,ZANDAKA_LAST_TOZA_201501,ZANDAKA_LAST_TOZA_201502,ZANDAKA_LAST_TOZA_201503,ZANDAKA_LAST_TOZA_201504,ZANDAKA_LAST_TOZA_201505,ZANDAKA_LAST_TOZA_201506,ZANDAKA_LAST_TOZA_201507,ZANDAKA_LAST_TOZA_201508,ZANDAKA_LAST_TOZA_201509,ZANDAKA_LAST_TOZA_201510,ZANDAKA_LAST_TOZA_201511,ZANDAKA_LAST_TOZA_201512,ZANDAKA_LAST_TOZA_201601,ZANDAKA_LAST_TOZA_201602,ZANDAKA_LAST_TOZA_201603,ZANDAKA_LAST_TOZA_201604,ZANDAKA_LAST_TOZA_201605,ZANDAKA_LAST_TOZA_201606,ZANDAKA_LAST_TOZA_201607,ZANDAKA_LAST_TOZA_201608,ZANDAKA_LAST_TOZA_201609,ZANDAKA_LAST_TOZA_201610,ZANDAKA_LAST_TOZA_201611,ZANDAKA_LAST_TOZA_201612,ZANDAKA_LAST_TOZA_201701,ZANDAKA_LAST_TOZA_201702,ZANDAKA_LAST_TOZA_201703,ZANDAKA_LAST_TOZA_201704,ZANDAKA_LAST_TOZA_201705,ZANDAKA_LAST_TOZA_201706,ZANDAKA_LAST_TOZA_201707,ZANDAKA_LAST_TOZA_201708,ZANDAKA_LAST_TOZA_201709,ZANDAKA_LAST_TOZA_201710,ZANDAKA_LAST_TOZA_201711,ZANDAKA_LAST_TOZA_201712,ZANDAKA_LAST_TOZA_201801,ZANDAKA_LAST_TOZA_201802,ZANDAKA_LAST_TOZA_201803
0,6316050312,128444458.0,119160459.0,197201320.0,75066719.0,65203443.0,157908152.0,220937457.0,161375352.0,198614315.0,495257154.0,321075134.0,429730618.0,183383852.0,128572095.0,22702356.0,141643079.0,267821635.0,176160512.0,173112393.0,85045780.0,354193969.0,141999831.0,146807567.0,158034420.0,379712248.0,240395049.0,139795201.0,289020895.0,128317723.0,55174921.0,129188080.0,57865301.0,185345576.0,96686815.0,148763805.0,153550505.0,294528155.0,107644467.0,240129772.0,176544730.0,79187423.0,120592806.0,111810817.0,147349727.0,203067321.0,166769023.0,141594185.0,155002932.0
1,6316050371,96561700.0,83025322.0,15430755.0,20323254.0,39727290.0,24971763.0,9665376.0,71779893.0,140498868.0,44709512.0,22736033.0,38756122.0,33031202.0,50919863.0,30778664.0,52071185.0,30745985.0,62560873.0,80206262.0,30506012.0,26871966.0,19237593.0,23542803.0,88527989.0,51210533.0,189960069.0,137231191.0,209966961.0,83334852.0,5939231.0,14773803.0,82969676.0,12378001.0,32110831.0,44048761.0,64107668.0,41849122.0,27412485.0,56802355.0,87996082.0,23031605.0,179756436.0,13116863.0,29858768.0,33408207.0,28098473.0,24380940.0,130976677.0
2,6316050390,92867999.0,81851388.0,123468039.0,70587170.0,125371838.0,81991716.0,65187158.0,64690477.0,72534389.0,153400636.0,122227737.0,99672385.0,103228433.0,94361285.0,93096229.0,119293388.0,107025410.0,147762609.0,85262220.0,72778384.0,101935537.0,64136064.0,45938253.0,54107057.0,58434664.0,112991969.0,110960234.0,78732892.0,353940103.0,211056046.0,109041346.0,110585415.0,119714316.0,313832601.0,104015060.0,97533050.0,81299932.0,64940646.0,52675188.0,427824612.0,114988704.0,101862850.0,56775069.0,76013700.0,146571966.0,122978454.0,177930699.0,125239252.0
3,6316050420,253337502.0,283348532.0,268971265.0,267203353.0,265253130.0,242994892.0,254832707.0,261823202.0,241602459.0,255352347.0,279839633.0,285659119.0,296362579.0,302560207.0,292242496.0,289162278.0,273545490.0,265813876.0,305981532.0,267023739.0,261116958.0,270570983.0,288458207.0,304203732.0,336130560.0,328621975.0,347896653.0,296283230.0,311675683.0,323594420.0,321701559.0,325872302.0,356508783.0,353334698.0,360308415.0,379325453.0,403146619.0,392545279.0,403569378.0,409755731.0,385277749.0,402302256.0,376497628.0,361791896.0,352811673.0,403185014.0,315606614.0,86278023.0
4,6316050622,341876955.0,285731522.0,181790253.0,150901285.0,150901403.0,278947978.0,205678369.0,227341339.0,235895663.0,300162145.0,273243852.0,699682268.0,204059308.0,176373002.0,179116923.0,265815823.0,175437568.0,412467056.0,263709762.0,190641054.0,193180008.0,317222290.0,192918153.0,272343309.0,295609964.0,321273599.0,173693266.0,456785572.0,173935059.0,245572656.0,109621015.0,70494641.0,274514898.0,144601382.0,114090130.0,288713869.0,233831266.0,163749548.0,160972404.0,131829745.0,116576703.0,22935

## 指定期間での分割横持ち化

In [92]:
period_length = 12
months = np.sort(df_nyukin['YYYYMM'].unique())

In [93]:
# 　月末残高（普通・当座）指定期間集計
df_zandaka_last_monthly_periods_forplot  = apply_suffix(
    get_monthly_period(df_zandaka_last_monthly,  months, period_length, True), period_length, '月末残高_普通当座')

In [94]:
print(df_zandaka_last_monthly_periods_forplot.shape)
df_zandaka_last_monthly_periods_forplot.head()

(104004, 14)


,GODCUSTCD,YYYYMM,0_月末残高_普通当座,1_月末残高_普通当座,2_月末残高_普通当座,3_月末残高_普通当座,4_月末残高_普通当座,5_月末残高_普通当座,6_月末残高_普通当座,7_月末残高_普通当座,8_月末残高_普通当座,9_月末残高_普通当座,10_月末残高_普通当座,11_月末残高_普通当座
0,6316050312,201503,214497845.0,180583846.0,258781448.0,136271686.0,166971491.0,245896200.0,333885505.0,262205606.0,279639033.0,562457872.0,381366002.0,473741486.0
1,6316050371,201503,96661035.0,83123577.0,15527930.0,20419349.0,39823393.0,25067866.0,9761479.0,71875996.0,140594971.0,44805615.0,22832144.0,38852233.0
2,6316050390,201503,98331211.0,96057754.0,131224390.0,77097005.0,127660180.0,124779822.0,69034347.0,66706756.0,76106262.0,156095537.0,136891401.0,111597092.0
3,6316050420,201503,307852228.0,332209104.0,312887224.0,312948417.0,313061457.0,280519285.0,300487165.0,292940748.0,320613497.0,345065152.0,360806778.0,368172911.0
4,6316050622,201503,381101671.0,310390466.0,231805432.0,186006288.0,172915026.0,311031998.0,255089466.0,255291955.0,258303654.0,323289283.0,296469160.0,735120019.0


In [95]:
# 　月末残高（普通）指定期間集計
df_zandaka_last_futu_monthly_periods_forplot  = apply_suffix(
    get_monthly_period(df_zandaka_last_futu_monthly,  months, period_length, True), period_length, '月末残高_普通')

In [96]:
print(df_zandaka_last_futu_monthly_periods_forplot.shape)
df_zandaka_last_futu_monthly_periods_forplot.head()

(94536, 14)


,GODCUSTCD,YYYYMM,0_月末残高_普通,1_月末残高_普通,2_月末残高_普通,3_月末残高_普通,4_月末残高_普通,5_月末残高_普通,6_月末残高_普通,7_月末残高_普通,8_月末残高_普通,9_月末残高_普通,10_月末残高_普通,11_月末残高_普通
0,6316050312,201503,86053387.0,61423387.0,61580128.0,61204967.0,101768048.0,87988048.0,112948048.0,100830254.0,81024718.0,67200718.0,60290868.0,44010868.0
1,6316050371,201503,99335.0,98255.0,97175.0,96095.0,96103.0,96103.0,96103.0,96103.0,96103.0,96103.0,96111.0,96111.0
2,6316050390,201503,5463212.0,14206366.0,7756351.0,6509835.0,2288342.0,42788106.0,3847189.0,2016279.0,3571873.0,2694901.0,14663664.0,11924707.0
3,6316050420,201503,54514726.0,48860572.0,43915959.0,45745064.0,47808327.0,37524393.0,45654458.0,31117546.0,79011038.0,89712805.0,80967145.0,82513792.0
4,6316050622,201503,39224716.0,24658944.0,50015179.0,35105003.0,22013623.0,32084020.0,49411097.0,27950616.0,22407991.0,23127138.0,23225308.0,35437751.0


In [97]:
# 正規化
df_zandaka_last_futu_monthly_periods_forplot_norm = normalization(df_zandaka_last_futu_monthly_periods_forplot)
print(df_zandaka_last_futu_monthly_periods_forplot_norm.shape)

(94536, 14)


In [98]:
# 標準化
df_zandaka_last_futu_monthly_periods_forplot_stand = standardization(df_zandaka_last_futu_monthly_periods_forplot)
print(df_zandaka_last_futu_monthly_periods_forplot_stand.shape)

(94536, 14)


In [99]:
# 　月末残高（当座）指定期間集計
df_zandaka_last_toza_monthly_periods_forplot  = apply_suffix(
    get_monthly_period(df_zandaka_last_toza_monthly,  months, period_length, True), period_length, '月末残高_当座')

In [100]:
print(df_zandaka_last_toza_monthly_periods_forplot.shape)
df_zandaka_last_toza_monthly_periods_forplot.head()

(30312, 14)


,GODCUSTCD,YYYYMM,0_月末残高_当座,1_月末残高_当座,2_月末残高_当座,3_月末残高_当座,4_月末残高_当座,5_月末残高_当座,6_月末残高_当座,7_月末残高_当座,8_月末残高_当座,9_月末残高_当座,10_月末残高_当座,11_月末残高_当座
0,6316050312,201503,128444458.0,119160459.0,197201320.0,75066719.0,65203443.0,157908152.0,220937457.0,161375352.0,198614315.0,495257154.0,321075134.0,429730618.0
1,6316050371,201503,96561700.0,83025322.0,15430755.0,20323254.0,39727290.0,24971763.0,9665376.0,71779893.0,140498868.0,44709512.0,22736033.0,38756122.0
2,6316050390,201503,92867999.0,81851388.0,123468039.0,70587170.0,125371838.0,81991716.0,65187158.0,64690477.0,72534389.0,153400636.0,122227737.0,99672385.0
3,6316050420,201503,253337502.0,283348532.0,268971265.0,267203353.0,265253130.0,242994892.0,254832707.0,261823202.0,241602459.0,255352347.0,279839633.0,285659119.0
4,6316050622,201503,341876955.0,285731522.0,181790253.0,150901285.0,150901403.0,278947978.0,205678369.0,227341339.0,235895663.0,300162145.0,273243852.0,699682268.0


## 年次集計値の作成

In [101]:
# 年次集計の設定値
period_length = 12
months = np.sort(df_shukkin['YYYYMM'].unique())
CUST_KEY = ['GODCUSTCD']
L = len(list(CUST_KEY))

In [102]:
# 月末残高（普通・当座）の年次集計値
df_zandaka_last_freq  = year_aggregation_simple(df_zandaka_last_monthly, period_length, months, CUST_KEY, L)
print(df_zandaka_last_freq.shape)
df_zandaka_last_freq.head()

(104004, 3)


,GODCUSTCD,YYYYMM,ZANDAKA_LAST_MEAN12
0,6316050312,201503,2.913582e+08
1,6316050371,201503,5.077880e+07
2,6316050390,201503,1.059651e+08
3,6316050420,201503,3.206303e+08
4,6316050622,201503,3.097345e+08


In [103]:
# 月末残高（普通）の年次集計値
df_zandaka_last_futu_freq  = year_aggregation_simple(df_zandaka_last_futu_monthly, period_length, months, CUST_KEY, L)

In [104]:
print(df_zandaka_last_futu_freq.shape)
df_zandaka_last_futu_freq.head()

(94536, 3)


,GODCUSTCD,YYYYMM,ZANDAKA_LAST_FUTU_MEAN12
0,6316050312,201503,7.719362e+07
1,6316050371,201503,9.664167e+04
2,6316050390,201503,9.810902e+06
3,6316050420,201503,5.727882e+07
4,6316050622,201503,3.205512e+07


In [105]:
# # 月末残高（普通）の年次集計値（正規化）
# df_zandaka_last_futu_freq_norm  = year_aggregation_simple(df_zandaka_last_futu_monthly_norm, period_length, months, CUST_KEY, L)

In [106]:
# print(df_zandaka_last_futu_freq_norm.shape)
# df_zandaka_last_futu_freq_norm.head()

In [107]:
# # 月末残高（普通）の年次集計値（標準化）
# df_zandaka_last_futu_freq_stand  = year_aggregation_simple(df_zandaka_last_futu_monthly_stand, period_length, months, CUST_KEY, L)

In [108]:
# print(df_zandaka_last_futu_freq_stand.shape)
# df_zandaka_last_futu_freq_stand.head()

In [109]:
# 月末残高（当座）の年次集計値
df_zandaka_last_toza_freq  = year_aggregation_simple(df_zandaka_last_toza_monthly, period_length, months, CUST_KEY, L)

In [110]:
print(df_zandaka_last_toza_freq.shape)
df_zandaka_last_toza_freq.head()

(30312, 3)


,GODCUSTCD,YYYYMM,ZANDAKA_LAST_TOZA_MEAN12
0,6316050312,201503,2.141645e+08
1,6316050371,201503,5.068216e+07
2,6316050390,201503,9.615424e+07
3,6316050420,201503,2.633515e+08
4,6316050622,201503,2.776794e+08


## 月次為替データの取得

In [111]:
# 入金データの取得
df_kawase_nyukin = godcustcd_preprocessing(pd.read_csv(PATH_INPUT+'nyukin_kawase_monthly.csv', index_col=0))

In [112]:
print(df_kawase_nyukin.shape)
df_kawase_nyukin.sort_values(['GODCUSTCD','YYYYMM']).head()

(73146, 5)


,GODCUSTCD,YYYYMM,AMOUNT,COUNT,PAY_OR_RECEIVE
111,6316050371,201404,160374686,100,1
112,6316050371,201405,140195677,60,1
113,6316050371,201406,46001020,58,1
114,6316050371,201407,144833698,54,1
115,6316050371,201408,107588411,75,1


In [113]:
# 出金データの取得
df_kawase_shukkin = godcustcd_preprocessing(pd.read_csv(PATH_INPUT+'shukkin_kawase_monthly.csv', index_col=0))

In [114]:
print(df_kawase_shukkin.shape)
df_kawase_shukkin.sort_values(['GODCUSTCD','YYYYMM']).head()

(48987, 5)


,GODCUSTCD,YYYYMM,AMOUNT,COUNT,PAY_OR_RECEIVE
49,6316050371,201404,76717638,51,0
50,6316050371,201405,121916441,50,0
51,6316050371,201406,83046671,59,0
52,6316050371,201407,93941975,44,0
53,6316050371,201408,49909073,41,0


## 月次為替データの横持ち化

In [115]:
CUST_KEY = ['GODCUSTCD'] 

In [116]:
# 　入金額横持ち
df_cust_master = pd.DataFrame(df_kawase_nyukin[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_kawase_nyukin_amount_monthly  = get_monthly_yokomochi(
    df_cust_master.copy(), df_kawase_nyukin[df_kawase_nyukin['PAY_OR_RECEIVE']==1], 'YYYYMM', 'AMOUNT', 'RECEIVE')

In [117]:
print(df_kawase_nyukin_amount_monthly.shape)
df_kawase_nyukin_amount_monthly.head()

(2306, 49)


,GODCUSTCD,RECEIVE_AMOUNT_201404,RECEIVE_AMOUNT_201405,RECEIVE_AMOUNT_201406,RECEIVE_AMOUNT_201407,RECEIVE_AMOUNT_201408,RECEIVE_AMOUNT_201409,RECEIVE_AMOUNT_201410,RECEIVE_AMOUNT_201411,RECEIVE_AMOUNT_201412,RECEIVE_AMOUNT_201501,RECEIVE_AMOUNT_201502,RECEIVE_AMOUNT_201503,RECEIVE_AMOUNT_201504,RECEIVE_AMOUNT_201505,RECEIVE_AMOUNT_201506,RECEIVE_AMOUNT_201507,RECEIVE_AMOUNT_201508,RECEIVE_AMOUNT_201509,RECEIVE_AMOUNT_201510,RECEIVE_AMOUNT_201511,RECEIVE_AMOUNT_201512,RECEIVE_AMOUNT_201601,RECEIVE_AMOUNT_201602,RECEIVE_AMOUNT_201603,RECEIVE_AMOUNT_201604,RECEIVE_AMOUNT_201605,RECEIVE_AMOUNT_201606,RECEIVE_AMOUNT_201607,RECEIVE_AMOUNT_201608,RECEIVE_AMOUNT_201609,RECEIVE_AMOUNT_201610,RECEIVE_AMOUNT_201611,RECEIVE_AMOUNT_201612,RECEIVE_AMOUNT_201701,RECEIVE_AMOUNT_201702,RECEIVE_AMOUNT_201703,RECEIVE_AMOUNT_201704,RECEIVE_AMOUNT_201705,RECEIVE_AMOUNT_201706,RECEIVE_AMOUNT_201707,RECEIVE_AMOUNT_201708,RECEIVE_AMOUNT_201709,RECEIVE_AMOUNT_201710,RECEIVE_AMOUNT_201711,RECEIVE_AMOUNT_201712,RECEIVE_AMOUNT_201801,RECEIVE_AMOUNT_201802,RECEIVE_AMOUNT_201803
0,6316050371,1.603747e+08,140195677.0,46001020.0,144833698.0,107588411.0,1.324518e+08,176698655.0,151786845.0,147700118.0,147068225.0,4.516709e+07,126940905.0,168062883.0,172100701.0,99170886.0,173312272.0,76482003.0,309688884.0,114029398.0,20420833.0,95237137.0,47893987.0,96490950.0,253256109.0,275088640.0,293711326.0,22969111.0,184796845.0,26950649.0,49064335.0,84211467.0,116959995.0,48042300.0,160194731.0,79275741.0,178918456.0,113145012.0,131324376.0,212144329.0,152537390.0,57809837.0,323022699.0,46734098.0,69748518.0,70857345.0,8.783129e+07,72774793.0,265492432.0
1,6316050390,4.033789e+07,145935126.0,63920988.0,17500675.0,123397950.0,7.611710e+07,33729010.0,28690282.0,64882144.0,187442661.0,1.315196e+08,33332912.0,48780887.0,180911317.0,103646519.0,127805826.0,54785859.0,43466556.0,84582703.0,95481863.0,55078237.0,37796005.0,36449534.0,114696684.0,42714012.0,79291486.0,55429863.0,22594721.0,39405008.0,75309360.0,14906224.0,35488383.0,69227853.0,38445208.0,32356835.0,44931886.0,32002807.0,69399001.0,32287839.0,61070742.0,24026183.0,50925420.0,73201940.0,34564581.0,105546019.0,1.293290e+08,71839699.0,38234005.0
2,6316050420,6.296667e+07,53540388.0,53566610.0,44237621.0,48346455.0,4.987043e+07,44408182.0,44309997.0,84767408.0,87344277.0,8.615307e+07,84453871.0,73990384.0,61101288.0,67854522.0,78180838.0,58746029.0,83191490.0,58920991.0,61157026.0,56841873.0,61256545.0,57098440.0,55370743.0,40243958.0,42695130.0,40481157.0,36707494.0,44862376.0,43596589.0,35143310.0,54849047.0,39475268.0,42964485.0,40739452.0,56603532.0,47099587.0,48665634.0,39068451.0,70642916.0,37103514.0,33786658.0,37459873.0,43654188.0,40876003.0,1.239613e+08,137571160.0,51418369.0
3,6316050622,1.113283e+09,760883731.0,863901014.0,907319201.0,733220966.0,1.040850e+09,850291676.0,856255461.0,956364551.0,924178923.0,1.304996e+09,896416127.0,864566346.0,639337409.0,811726422.0,766658103.0,642556979.0,686957878.0,749017703.0,597667776.0,773126055.0,841637518.0,802840591.0,657910706.0,529837685.0,746368403.0,685351097.0,632088767.0,686470017.0,503940660.0,579209434.0,664427928.0,871360832.0,790846991.0,836975485.0,818489934.0,714876041.0,764387601.0,773175621.0,639656408.0,607981191.0,661313700.0,715497948.0,869232887.0,924941188.0,1.323186e+09,960418336.0,834841702.0
4,6316050673,9.700000e+05,940000.0,935000.0,950000.0,930000.0,9.500000e+05,950000.0,900000.0,940000.0,920000.0,9.000000e+05,940000.0,930000.0,900000.0,970000.0,840000.0,950000.0,900000.0,1000000.0,800000.0,900000.0,900000.0,900000.0,850000.0,1000000.0,800000.0,850000.0,880000.0,2483000.0,NaN,250000.0,900000.0,930000.0,950000.0,920000.0,920000.0,950000.0,910000.0,920000.0,920000.0,920000.0,920000.0,920000.0,NaN,900000.0,9.500000e+05,910000.0,950000.0


In [118]:
# 　出金額横持ち
df_cust_master = pd.DataFrame(df_kawase_shukkin[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_kawase_shukkin_amount_monthly  = get_monthly_yokomochi(
    df_cust_master.copy(), df_kawase_shukkin[df_kawase_shukkin['PAY_OR_RECEIVE']==0], 'YYYYMM', 'AMOUNT', 'PAY')

In [119]:
print(df_kawase_shukkin_amount_monthly.shape)
df_kawase_shukkin_amount_monthly.head()

(2034, 49)


,GODCUSTCD,PAY_AMOUNT_201404,PAY_AMOUNT_201405,PAY_AMOUNT_201406,PAY_AMOUNT_201407,PAY_AMOUNT_201408,PAY_AMOUNT_201409,PAY_AMOUNT_201410,PAY_AMOUNT_201411,PAY_AMOUNT_201412,PAY_AMOUNT_201501,PAY_AMOUNT_201502,PAY_AMOUNT_201503,PAY_AMOUNT_201504,PAY_AMOUNT_201505,PAY_AMOUNT_201506,PAY_AMOUNT_201507,PAY_AMOUNT_201508,PAY_AMOUNT_201509,PAY_AMOUNT_201510,PAY_AMOUNT_201511,PAY_AMOUNT_201512,PAY_AMOUNT_201601,PAY_AMOUNT_201602,PAY_AMOUNT_201603,PAY_AMOUNT_201604,PAY_AMOUNT_201605,PAY_AMOUNT_201606,PAY_AMOUNT_201607,PAY_AMOUNT_201608,PAY_AMOUNT_201609,PAY_AMOUNT_201610,PAY_AMOUNT_201611,PAY_AMOUNT_201612,PAY_AMOUNT_201701,PAY_AMOUNT_201702,PAY_AMOUNT_201703,PAY_AMOUNT_201704,PAY_AMOUNT_201705,PAY_AMOUNT_201706,PAY_AMOUNT_201707,PAY_AMOUNT_201708,PAY_AMOUNT_201709,PAY_AMOUNT_201710,PAY_AMOUNT_201711,PAY_AMOUNT_201712,PAY_AMOUNT_201801,PAY_AMOUNT_201802,PAY_AMOUNT_201803
0,6316050371,7.671764e+07,1.219164e+08,8.304667e+07,93941975.0,49909073.0,1.210984e+08,1.589959e+08,9.777061e+07,3.977094e+07,222216984.0,4.419915e+07,72467183.0,1.403682e+08,125491123.0,92770346.0,84365467.0,66093592.0,354010867.0,49855584.0,47080323.0,69447519.0,9783809.0,51614443.0,216930036.0,282200265.0,68740874.0,142795111.0,72505748.0,36147914.0,54754728.0,34724100.0,29940494.0,93822024.0,81338238.0,42181801.0,132426433.0,102752510.0,113480493.0,145756113.0,84857133.0,90331181.0,137489054.0,179958874.0,29412887.0,4.437879e+07,3.770521e+07,5.584201e+07,139914908.0
1,6316050390,7.322952e+07,8.793753e+07,7.233880e+07,135694998.0,99422670.0,6.007215e+07,8.452835e+07,1.001544e+08,8.706851e+07,60041142.0,1.085848e+08,112898575.0,5.986889e+07,66959066.0,57367561.0,44197028.0,54593872.0,69368246.0,59953741.0,83385812.0,57211501.0,77274330.0,61114220.0,68253534.0,52322534.0,50739653.0,57414103.0,97109874.0,97944614.0,101674251.0,134150847.0,132726696.0,194345797.0,140050338.0,108867674.0,99965162.0,77595101.0,62119266.0,79172695.0,70643104.0,65804194.0,56172830.0,54129084.0,54657124.0,5.368922e+07,5.540878e+07,6.052791e+07,43592951.0
2,6316050420,6.639591e+07,6.219879e+07,5.517208e+07,60861486.0,59795268.0,4.967434e+07,4.857142e+07,5.483567e+07,7.981733e+07,91965003.0,7.991408e+07,69556044.0,7.569612e+07,74418162.0,69733353.0,76185206.0,68972850.0,66599069.0,63844290.0,83601309.0,53575451.0,55521489.0,30295708.0,39968973.0,29869561.0,32886489.0,28979794.0,39854300.0,34153448.0,27777775.0,25669145.0,22902176.0,33080426.0,36717621.0,32128576.0,36616357.0,37635530.0,32337305.0,31075188.0,48956614.0,32750854.0,28782323.0,40874520.0,37830685.0,3.850591e+07,2.391499e+07,1.975218e+08,331268821.0
3,6316050622,1.162230e+09,1.058643e+09,1.194702e+09,945159011.0,928000938.0,1.023185e+09,1.055184e+09,1.178734e+09,1.088941e+09,933696893.0,1.527624e+09,880273504.0,1.060209e+09,811758499.0,838995289.0,739335625.0,670667691.0,614205869.0,805643246.0,902001567.0,687654828.0,746289696.0,801594874.0,727929127.0,611115486.0,681872970.0,804709970.0,686449468.0,542977869.0,568403537.0,698087052.0,790909474.0,902485753.0,940206269.0,915792395.0,782809156.0,813234650.0,702506715.0,760129886.0,576927237.0,617154764.0,754366610.0,890047165.0,975828633.0,1.089839e+09,1.158989e+09,1.061951e+09,923751976.0
4,6316050673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9900000.0,NaN,NaN,NaN,NaN,NaN


In [120]:
# 　入金件数横持ち
df_cust_master = pd.DataFrame(df_kawase_nyukin[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_kawase_nyukin_count_monthly  = get_monthly_yokomochi(
    df_cust_master.copy(), df_kawase_nyukin[df_kawase_nyukin['PAY_OR_RECEIVE']==1], 'YYYYMM', 'COUNT', 'RECEIVE')

In [121]:
print(df_kawase_nyukin_count_monthly.shape)
df_kawase_nyukin_count_monthly.head()

(2306, 49)


,GODCUSTCD,RECEIVE_COUNT_201404,RECEIVE_COUNT_201405,RECEIVE_COUNT_201406,RECEIVE_COUNT_201407,RECEIVE_COUNT_201408,RECEIVE_COUNT_201409,RECEIVE_COUNT_201410,RECEIVE_COUNT_201411,RECEIVE_COUNT_201412,RECEIVE_COUNT_201501,RECEIVE_COUNT_201502,RECEIVE_COUNT_201503,RECEIVE_COUNT_201504,RECEIVE_COUNT_201505,RECEIVE_COUNT_201506,RECEIVE_COUNT_201507,RECEIVE_COUNT_201508,RECEIVE_COUNT_201509,RECEIVE_COUNT_201510,RECEIVE_COUNT_201511,RECEIVE_COUNT_201512,RECEIVE_COUNT_201601,RECEIVE_COUNT_201602,RECEIVE_COUNT_201603,RECEIVE_COUNT_201604,RECEIVE_COUNT_201605,RECEIVE_COUNT_201606,RECEIVE_COUNT_201607,RECEIVE_COUNT_201608,RECEIVE_COUNT_201609,RECEIVE_COUNT_201610,RECEIVE_COUNT_201611,RECEIVE_COUNT_201612,RECEIVE_COUNT_201701,RECEIVE_COUNT_201702,RECEIVE_COUNT_201703,RECEIVE_COUNT_201704,RECEIVE_COUNT_201705,RECEIVE_COUNT_201706,RECEIVE_COUNT_201707,RECEIVE_COUNT_201708,RECEIVE_COUNT_201709,RECEIVE_COUNT_201710,RECEIVE_COUNT_201711,RECEIVE_COUNT_201712,RECEIVE_COUNT_201801,RECEIVE_COUNT_201802,RECEIVE_COUNT_201803
0,6316050371,100.0,60.0,58.0,54.0,75.0,77.0,72.0,67.0,59.0,50.0,62.0,77.0,86.0,64.0,55.0,51.0,65.0,54.0,60.0,61.0,61.0,51.0,48.0,67.0,89.0,52.0,46.0,42.0,51.0,56.0,49.0,59.0,44.0,59.0,50.0,52.0,69.0,48.0,39.0,44.0,56.0,61.0,47.0,55.0,54.0,63.0,46.0,55.0
1,6316050390,47.0,55.0,64.0,50.0,50.0,56.0,54.0,47.0,51.0,45.0,47.0,49.0,48.0,41.0,53.0,51.0,48.0,44.0,36.0,52.0,52.0,48.0,40.0,53.0,46.0,51.0,46.0,40.0,42.0,48.0,36.0,37.0,44.0,45.0,31.0,48.0,38.0,48.0,43.0,43.0,40.0,40.0,48.0,47.0,51.0,40.0,37.0,44.0
2,6316050420,300.0,281.0,294.0,276.0,262.0,292.0,281.0,256.0,316.0,276.0,278.0,320.0,290.0,277.0,296.0,281.0,279.0,281.0,276.0,273.0,293.0,284.0,308.0,316.0,290.0,308.0,301.0,273.0,288.0,282.0,283.0,266.0,285.0,285.0,285.0,310.0,288.0,274.0,281.0,276.0,276.0,261.0,277.0,263.0,274.0,280.0,284.0,296.0
3,6316050622,796.0,708.0,782.0,753.0,718.0,758.0,746.0,711.0,836.0,844.0,783.0,854.0,802.0,710.0,779.0,762.0,747.0,733.0,713.0,771.0,795.0,785.0,812.0,804.0,746.0,775.0,752.0,722.0,774.0,707.0,743.0,750.0,784.0,793.0,764.0,803.0,726.0,798.0,758.0,762.0,762.0,697.0,766.0,761.0,780.0,832.0,820.0,760.0
4,6316050673,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0


In [122]:
# 　出金件数額横持ち
df_cust_master = pd.DataFrame(df_kawase_shukkin[CUST_KEY].drop_duplicates(), columns=[CUST_KEY])
df_kawase_shukkin_count_monthly  = get_monthly_yokomochi(
    df_cust_master.copy(), df_kawase_shukkin[df_kawase_shukkin['PAY_OR_RECEIVE']==0], 'YYYYMM', 'COUNT', 'PAY')

In [123]:
print(df_kawase_shukkin_count_monthly.shape)
df_kawase_shukkin_count_monthly.head()

(2034, 49)


,GODCUSTCD,PAY_COUNT_201404,PAY_COUNT_201405,PAY_COUNT_201406,PAY_COUNT_201407,PAY_COUNT_201408,PAY_COUNT_201409,PAY_COUNT_201410,PAY_COUNT_201411,PAY_COUNT_201412,PAY_COUNT_201501,PAY_COUNT_201502,PAY_COUNT_201503,PAY_COUNT_201504,PAY_COUNT_201505,PAY_COUNT_201506,PAY_COUNT_201507,PAY_COUNT_201508,PAY_COUNT_201509,PAY_COUNT_201510,PAY_COUNT_201511,PAY_COUNT_201512,PAY_COUNT_201601,PAY_COUNT_201602,PAY_COUNT_201603,PAY_COUNT_201604,PAY_COUNT_201605,PAY_COUNT_201606,PAY_COUNT_201607,PAY_COUNT_201608,PAY_COUNT_201609,PAY_COUNT_201610,PAY_COUNT_201611,PAY_COUNT_201612,PAY_COUNT_201701,PAY_COUNT_201702,PAY_COUNT_201703,PAY_COUNT_201704,PAY_COUNT_201705,PAY_COUNT_201706,PAY_COUNT_201707,PAY_COUNT_201708,PAY_COUNT_201709,PAY_COUNT_201710,PAY_COUNT_201711,PAY_COUNT_201712,PAY_COUNT_201801,PAY_COUNT_201802,PAY_COUNT_201803
0,6316050371,51.0,50.0,59.0,44.0,41.0,44.0,45.0,50.0,53.0,42.0,45.0,50.0,48.0,48.0,78.0,48.0,42.0,47.0,42.0,45.0,67.0,42.0,41.0,53.0,46.0,56.0,87.0,67.0,44.0,45.0,45.0,39.0,62.0,38.0,44.0,49.0,48.0,49.0,79.0,49.0,44.0,41.0,60.0,54.0,83.0,53.0,45.0,60.0
1,6316050390,123.0,144.0,136.0,148.0,139.0,131.0,135.0,147.0,143.0,133.0,141.0,129.0,127.0,132.0,142.0,133.0,135.0,132.0,127.0,137.0,134.0,145.0,128.0,140.0,122.0,125.0,134.0,138.0,137.0,133.0,151.0,143.0,155.0,138.0,135.0,133.0,130.0,132.0,135.0,142.0,141.0,128.0,129.0,119.0,119.0,123.0,121.0,121.0
2,6316050420,146.0,140.0,138.0,176.0,147.0,140.0,127.0,145.0,172.0,142.0,127.0,127.0,164.0,135.0,151.0,182.0,132.0,147.0,145.0,140.0,110.0,107.0,83.0,95.0,99.0,106.0,113.0,114.0,101.0,102.0,92.0,96.0,106.0,95.0,89.0,91.0,87.0,97.0,94.0,104.0,96.0,92.0,92.0,106.0,105.0,110.0,129.0,118.0
3,6316050622,416.0,446.0,455.0,428.0,387.0,403.0,408.0,417.0,486.0,390.0,404.0,392.0,406.0,385.0,451.0,425.0,414.0,418.0,419.0,425.0,477.0,420.0,406.0,396.0,398.0,418.0,490.0,444.0,397.0,404.0,412.0,399.0,461.0,395.0,398.0,385.0,395.0,395.0,462.0,432.0,416.0,397.0,413.0,429.0,505.0,467.0,467.0,448.0
4,6316050673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


## 指定期間での分割横持ち化

In [124]:
period_length = 12
months = np.sort(df_kawase_nyukin['YYYYMM'].unique())

In [125]:
df_kawase_nyukin_amount_monthly_periods_forplot  = apply_suffix(get_monthly_period(
    df_kawase_nyukin_amount_monthly,  months, period_length, True), period_length, '為替_入金_金額')

In [126]:
print(df_kawase_nyukin_amount_monthly_periods_forplot.shape)
df_kawase_nyukin_amount_monthly_periods_forplot.head()

(83016, 14)


,GODCUSTCD,YYYYMM,0_為替_入金_金額,1_為替_入金_金額,2_為替_入金_金額,3_為替_入金_金額,4_為替_入金_金額,5_為替_入金_金額,6_為替_入金_金額,7_為替_入金_金額,8_為替_入金_金額,9_為替_入金_金額,10_為替_入金_金額,11_為替_入金_金額
0,6316050371,201503,1.603747e+08,140195677.0,46001020.0,144833698.0,107588411.0,1.324518e+08,176698655.0,151786845.0,147700118.0,147068225.0,4.516709e+07,126940905.0
1,6316050390,201503,4.033789e+07,145935126.0,63920988.0,17500675.0,123397950.0,7.611710e+07,33729010.0,28690282.0,64882144.0,187442661.0,1.315196e+08,33332912.0
2,6316050420,201503,6.296667e+07,53540388.0,53566610.0,44237621.0,48346455.0,4.987043e+07,44408182.0,44309997.0,84767408.0,87344277.0,8.615307e+07,84453871.0
3,6316050622,201503,1.113283e+09,760883731.0,863901014.0,907319201.0,733220966.0,1.040850e+09,850291676.0,856255461.0,956364551.0,924178923.0,1.304996e+09,896416127.0
4,6316050673,201503,9.700000e+05,940000.0,935000.0,950000.0,930000.0,9.500000e+05,950000.0,900000.0,940000.0,920000.0,9.000000e+05,940000.0


In [127]:
df_kawase_shukkin_amount_monthly_periods_forplot  = apply_suffix(get_monthly_period(
    df_kawase_shukkin_amount_monthly,  months, period_length, True), period_length, '為替_出金_金額')

In [128]:
print(df_kawase_shukkin_amount_monthly_periods_forplot.shape)
df_kawase_shukkin_amount_monthly_periods_forplot.head()

(73224, 14)


,GODCUSTCD,YYYYMM,0_為替_出金_金額,1_為替_出金_金額,2_為替_出金_金額,3_為替_出金_金額,4_為替_出金_金額,5_為替_出金_金額,6_為替_出金_金額,7_為替_出金_金額,8_為替_出金_金額,9_為替_出金_金額,10_為替_出金_金額,11_為替_出金_金額
0,6316050371,201503,7.671764e+07,1.219164e+08,8.304667e+07,93941975.0,49909073.0,1.210984e+08,1.589959e+08,9.777061e+07,3.977094e+07,222216984.0,4.419915e+07,72467183.0
1,6316050390,201503,7.322952e+07,8.793753e+07,7.233880e+07,135694998.0,99422670.0,6.007215e+07,8.452835e+07,1.001544e+08,8.706851e+07,60041142.0,1.085848e+08,112898575.0
2,6316050420,201503,6.639591e+07,6.219879e+07,5.517208e+07,60861486.0,59795268.0,4.967434e+07,4.857142e+07,5.483567e+07,7.981733e+07,91965003.0,7.991408e+07,69556044.0
3,6316050622,201503,1.162230e+09,1.058643e+09,1.194702e+09,945159011.0,928000938.0,1.023185e+09,1.055184e+09,1.178734e+09,1.088941e+09,933696893.0,1.527624e+09,880273504.0
4,6316050673,201503,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0


In [129]:
df_kawase_nyukin_count_monthly_periods_forplot  = apply_suffix(get_monthly_period(
    df_kawase_nyukin_count_monthly,  months, period_length, True), period_length, '為替_入金_件数')

In [130]:
print(df_kawase_nyukin_count_monthly_periods_forplot.shape)
df_kawase_nyukin_count_monthly_periods_forplot.head()

(83016, 14)


,GODCUSTCD,YYYYMM,0_為替_入金_件数,1_為替_入金_件数,2_為替_入金_件数,3_為替_入金_件数,4_為替_入金_件数,5_為替_入金_件数,6_為替_入金_件数,7_為替_入金_件数,8_為替_入金_件数,9_為替_入金_件数,10_為替_入金_件数,11_為替_入金_件数
0,6316050371,201503,100.0,60.0,58.0,54.0,75.0,77.0,72.0,67.0,59.0,50.0,62.0,77.0
1,6316050390,201503,47.0,55.0,64.0,50.0,50.0,56.0,54.0,47.0,51.0,45.0,47.0,49.0
2,6316050420,201503,300.0,281.0,294.0,276.0,262.0,292.0,281.0,256.0,316.0,276.0,278.0,320.0
3,6316050622,201503,796.0,708.0,782.0,753.0,718.0,758.0,746.0,711.0,836.0,844.0,783.0,854.0
4,6316050673,201503,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [131]:
df_kawase_shukkin_count_monthly_periods_forplot  = apply_suffix(get_monthly_period(
    df_kawase_shukkin_count_monthly,  months, period_length, True), period_length, '為替_出金_件数')

In [132]:
print(df_kawase_shukkin_count_monthly_periods_forplot.shape)
df_kawase_shukkin_count_monthly_periods_forplot.head()

(73224, 14)


,GODCUSTCD,YYYYMM,0_為替_出金_件数,1_為替_出金_件数,2_為替_出金_件数,3_為替_出金_件数,4_為替_出金_件数,5_為替_出金_件数,6_為替_出金_件数,7_為替_出金_件数,8_為替_出金_件数,9_為替_出金_件数,10_為替_出金_件数,11_為替_出金_件数
0,6316050371,201503,51.0,50.0,59.0,44.0,41.0,44.0,45.0,50.0,53.0,42.0,45.0,50.0
1,6316050390,201503,123.0,144.0,136.0,148.0,139.0,131.0,135.0,147.0,143.0,133.0,141.0,129.0
2,6316050420,201503,146.0,140.0,138.0,176.0,147.0,140.0,127.0,145.0,172.0,142.0,127.0,127.0
3,6316050622,201503,416.0,446.0,455.0,428.0,387.0,403.0,408.0,417.0,486.0,390.0,404.0,392.0
4,6316050673,201503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 年次集計値の作成

In [133]:
# 年次集計の設定値
period_length = 12
months = np.sort(df_kawase_nyukin['YYYYMM'].unique())
CUST_KEY = ['GODCUSTCD']
L = len(list(CUST_KEY))

In [134]:
# 入金データの年次集計値
df_kawase_nyukin_freq  = year_aggregation(
    df_kawase_nyukin_amount_monthly, df_kawase_nyukin_count_monthly,
    period_length, months, CUST_KEY, L
)
df_kawase_nyukin_freq = df_kawase_nyukin_freq.rename(columns=
                                                     {'RECEIVE_AMOUNT_MEAN12':'KAWASE_RECEIVE_AMOUNT_MEAN12',
                                                      'RECEIVE_COUNT_MEAN12':'KAWASE_RECEIVE_COUNT_MEAN12'})

In [135]:
print(df_kawase_nyukin_freq.shape)
df_kawase_nyukin_freq.head()

(83016, 4)


,GODCUSTCD,YYYYMM,KAWASE_RECEIVE_AMOUNT_MEAN12,KAWASE_RECEIVE_COUNT_MEAN12
0,6316050371,201503,1.272339e+08,67.583333
1,6316050390,201503,7.890053e+07,51.250000
2,6316050420,201503,6.199708e+07,286.000000
3,6316050622,201503,9.339968e+08,774.083333
4,6316050673,201503,9.354167e+05,1.000000


In [136]:
# 出金データの年次集計値
df_kawase_shukkin_freq  = year_aggregation(
    df_kawase_shukkin_amount_monthly, df_kawase_shukkin_count_monthly,
    period_length, months, CUST_KEY, L
)
df_kawase_shukkin_freq = df_kawase_shukkin_freq.rename(columns=
                                                       {'PAY_AMOUNT_MEAN12':'KAWASE_PAY_AMOUNT_MEAN12',
                                                        'PAY_COUNT_MEAN12':'KAWASE_PAY_COUNT_MEAN12'})

In [137]:
print(df_kawase_shukkin_freq.shape)
df_kawase_shukkin_freq.head()

(73224, 4)


,GODCUSTCD,YYYYMM,KAWASE_PAY_AMOUNT_MEAN12,KAWASE_PAY_COUNT_MEAN12
0,6316050371,201503,9.850425e+07,47.833333
1,6316050390,201503,9.016429e+07,137.416667
2,6316050420,201503,6.489645e+07,143.916667
3,6316050622,201503,1.081364e+09,419.333333
4,6316050673,201503,0.000000e+00,0.000000


## ファイル出力

In [139]:
# 入出金（普通・当座）関連
df_nyukin_amount_monthly_periods_forplot.to_csv(PATH_OUTPUT+'nyukin_amount_monthly_periods_forplot.csv')
df_nyukin_count_monthly_periods_forplot.to_csv(PATH_OUTPUT+'nyukin_count_monthly_periods_forplot.csv')
df_shukkin_amount_monthly_periods_forplot.to_csv(PATH_OUTPUT+'shukkin_amount_monthly_periods_forplot.csv')
df_shukkin_count_monthly_periods_forplot.to_csv(PATH_OUTPUT+'shukkin_count_monthly_periods_forplot.csv')
df_nyukin_freq.to_csv(PATH_OUTPUT+'nyukin_freq.csv')
df_shukkin_freq.to_csv(PATH_OUTPUT+'shukkin_freq.csv')
# 入出金（普通・当座）関連（正規化）
df_nyukin_amount_monthly_periods_forplot_norm.to_csv(PATH_OUTPUT+'nyukin_amount_monthly_periods_forplot_norm.csv')
df_nyukin_count_monthly_periods_forplot_norm.to_csv(PATH_OUTPUT+'nyukin_count_monthly_periods_forplot_norm.csv')
df_shukkin_amount_monthly_periods_forplot_norm.to_csv(PATH_OUTPUT+'shukkin_amount_monthly_periods_forplot_norm.csv')
df_shukkin_count_monthly_periods_forplot_norm.to_csv(PATH_OUTPUT+'shukkin_count_monthly_periods_forplot_norm.csv')
# df_nyukin_freq_norm.to_csv(PATH_OUTPUT+'nyukin_freq_norm.csv')
# df_shukkin_freq_norm.to_csv(PATH_OUTPUT+'shukkin_freq_norm.csv')
# 入出金（普通・当座）関連（標準化）
df_nyukin_amount_monthly_periods_forplot_stand.to_csv(PATH_OUTPUT+'nyukin_amount_monthly_periods_forplot_stand.csv')
df_nyukin_count_monthly_periods_forplot_stand.to_csv(PATH_OUTPUT+'nyukin_count_monthly_periods_forplot_stand.csv')
df_shukkin_amount_monthly_periods_forplot_stand.to_csv(PATH_OUTPUT+'shukkin_amount_monthly_periods_forplot_stand.csv')
df_shukkin_count_monthly_periods_forplot_stand.to_csv(PATH_OUTPUT+'shukkin_count_monthly_periods_forplot_stand.csv')
# df_nyukin_freq_stand.to_csv(PATH_OUTPUT+'nyukin_freq_stand.csv')
# df_shukkin_freq_stand.to_csv(PATH_OUTPUT+'shukkin_freq_stand.csv')
# 摘要込み入出金（普通・当座）関連
df_nyukin_tekiyo_amount_monthly_periods_forplot.to_csv(PATH_OUTPUT+'nyukin_tekiyo_amount_monthly_periods_forplot.csv')
df_nyukin_tekiyo_count_monthly_periods_forplot.to_csv(PATH_OUTPUT+'nyukin_tekiyo_count_monthly_periods_forplot.csv')
df_shukkin_tekiyo_amount_monthly_periods_forplot.to_csv(PATH_OUTPUT+'shukkin_tekiyo_amount_monthly_periods_forplot.csv')
df_shukkin_tekiyo_count_monthly_periods_forplot.to_csv(PATH_OUTPUT+'shukkin_tekiyo_count_monthly_periods_forplot.csv')
# 月末残高関連
df_zandaka_last_monthly_periods_forplot.to_csv(PATH_OUTPUT+'zandaka_last_monthly_periods_forplot.csv')
df_zandaka_last_futu_monthly_periods_forplot.to_csv(PATH_OUTPUT+'zandaka_last_futu_monthly_periods_forplot.csv')
df_zandaka_last_toza_monthly_periods_forplot.to_csv(PATH_OUTPUT+'zandaka_last_toza_monthly_periods_forplot.csv')
df_zandaka_last_freq.to_csv(PATH_OUTPUT+'zandaka_last_freq.csv')
df_zandaka_last_futu_freq.to_csv(PATH_OUTPUT+'zandaka_last_futu_freq.csv')
df_zandaka_last_toza_freq.to_csv(PATH_OUTPUT+'zandaka_last_toza_freq.csv')
# 月末残高関連（正規化）
df_zandaka_last_futu_monthly_periods_forplot_norm.to_csv(PATH_OUTPUT+'zandaka_last_futu_monthly_periods_forplot_norm.csv')
# df_zandaka_last_futu_freq_norm.to_csv(PATH_OUTPUT+'zandaka_last_futu_freq_norm.csv')
# 月末残高関連（標準化）
df_zandaka_last_futu_monthly_periods_forplot_stand.to_csv(PATH_OUTPUT+'zandaka_last_futu_monthly_periods_forplot_stand.csv')
# df_zandaka_last_futu_freq_stand.to_csv(PATH_OUTPUT+'zandaka_last_futu_freq_stand.csv')
# 入出金（為替）関連
df_kawase_nyukin_amount_monthly_periods_forplot.to_csv(PATH_OUTPUT+'kawase_nyukin_amount_monthly_periods_forplot.csv')
df_kawase_shukkin_amount_monthly_periods_forplot.to_csv(PATH_OUTPUT+'kawase_shukkin_amount_monthly_periods_forplot.csv')
df_kawase_nyukin_count_monthly_periods_forplot.to_csv(PATH_OUTPUT+'kawase_nyukin_count_monthly_periods_forplot.csv')
df_kawase_shukkin_count_monthly_periods_forplot.to_csv(PATH_OUTPUT+'kawase_shukkin_count_monthly_periods_forplot.csv')
df_kawase_nyukin_freq.to_csv(PATH_OUTPUT+'kawase_nyukin_freq.csv', index=False)
df_kawase_shukkin_freq.to_csv(PATH_OUTPUT+'kawase_shukkin_freq.csv', index=False)